# YOLO Hyperparameter Tuning

- Support for YOLOv8, YOLOv9, YOLOv10, YOLO11, YOLO12

In [1]:
# Base directories
# Detect environment: Colab or local

import os
from pathlib import Path


IS_COLAB = 'COLAB_GPU' in os.environ or os.path.exists('/content')

USE_WANDB = True  # Set to False to disable W&B logging



if IS_COLAB:
    #Mount Google Drive if not already mounted
    from google.colab import drive
    drive.mount('/content/Drive', force_remount=True)
    # Running in Google Colab
    BASE_DIR = Path('/content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo')

    # Configure W&B API key
    if USE_WANDB:
        # In Colab, get API key from secrets
        from google.colab import userdata
        wandb_api_key = userdata.get('wandb_api_key')
        os.environ['WANDB_API_KEY'] = wandb_api_key
        print('✓ W&B API key loaded from Colab secrets')

    DATASET_BASE_DIR = Path('/computer_vision_yolo')

else:
    # Running locally
    BASE_DIR = Path.cwd().parent
    if USE_WANDB:
        print('✓ Running locally - W&B will use existing login or prompt')

    DATASET_BASE_DIR = Path.cwd().parent


Mounted at /content/Drive
✓ W&B API key loaded from Colab secrets


In [2]:
# !cd /content/Drive/MyDrive/ksu_yolo_tuning_2025 && git clone https://github.com/m3mahdy/computer_vision_yolo

In [3]:
! cd {BASE_DIR} && pip install -r requirements.txt --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.7 MB/s eta 0:00:00


In [4]:
# limited dataset
!mkdir {DATASET_BASE_DIR}
!cd {BASE_DIR}/dataset && cp 8_download_extract_other_datasets.py {DATASET_BASE_DIR} && cd {DATASET_BASE_DIR} && python 8_download_extract_other_datasets.py



BDD100K DATASET DOWNLOAD & EXTRACT

Available datasets:

[1] Full Dataset Source Files
    Original BDD100K images and labels (requires processing)
    Size: ~5.28GB, ~180MB

[2] Limited Dataset (YOLO Ready)
    Balanced limited dataset - 30-40% coverage (~25K train)
    Size: ~3.3GB

[3] Tuning Dataset (YOLO Ready)
    Tuning dataset - 20% coverage (~14K train)
    Size: ~1.58GB

[4] Tiny Dataset (YOLO Ready)
    Tiny dataset - ~500 train, ~1K total (fast testing)
    Size: ~100MB

[5] Test Split Only (YOLO Ready)
    Test split only for validation (20K images)
    Size: ~1.1GB

[0] Exit

Select dataset (0-5): 3

Processing: Tuning Dataset (YOLO Ready)

📥 Downloading from Google Drive...
   File ID: 1QccaQ1tI_N3zXzp08Nemw13wl_02gUWG
   Destination: bdd100k_yolo_tuning.zip
Downloading...
From (original): https://drive.google.com/uc?id=1QccaQ1tI_N3zXzp08Nemw13wl_02gUWG
From (redirected): https://drive.google.com/uc?id=1QccaQ1tI_N3zXzp08Nemw13wl_02gUWG&confirm=t&uuid=e0e0eb62-4de6-44d2-

## 1. Import Required Libraries

In [5]:
# Install required libraries (uncomment if running in Colab)
# !pip install -q ultralytics optuna plotly kaleido wandb pyyaml

import os
import sys
import gc
import yaml
import json
import torch
import shutil
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from tqdm import tqdm
import pickle
import platform
import psutil

import wandb

# YOLO and Optuna imports
from ultralytics import YOLO
import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice

# ReportLab imports for PDF generation
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors as rl_colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_LEFT
from PIL import Image as PILImage

warnings.filterwarnings('ignore')

# Configure matplotlib for notebook display
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (15, 10)

# Check GPU availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'✓ Libraries imported successfully')
print(f'✓ Device: {device}')
if device == 'cuda':
    print(f'  GPU: {torch.cuda.get_device_name(0)}')
    print(f'  CUDA Version: {torch.version.cuda}')
    print(f'  Available Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✓ Libraries imported successfully
✓ Device: cuda
  GPU: NVIDIA A100-SXM4-40GB
  CUDA Version: 12.6
  Available Memory: 42.47 GB


## 2. Constants and Enums

In [6]:
# ============================================================================
# CONSTANTS AND ENUMS
# ============================================================================

class TrialStatus:
    """Constants for trial execution status"""
    COMPLETED = "completed"
    FAILED = "failed"
    PRUNED = "pruned"
    RUNNING = "running"

class DatasetSplit:
    """Constants for dataset split names"""
    TRAIN = "train"
    VAL = "val"
    TEST = "test"

class ModelConfig:
    """Default model training configuration constants"""
    # Training workers
    DEFAULT_WORKERS = 8  # Number of data loading workers

    # Early stopping and checkpointing
    DEFAULT_PATIENCE = 20  # Epochs to wait before early stopping

    # Augmentation timing
    CLOSE_MOSAIC_EPOCHS = 10  # Disable mosaic augmentation in last N epochs

print('✓ Constants and enums defined')

✓ Constants and enums defined


## 3. Configuration

In [7]:
# CONFIGURATION
# ============================================================================


# Model Selection - Choose one of the following:
MODEL_NAME = "yolov8m_finetuned_1"

#yolov10n is for testing purpose only
#Mahdy will work yolov8m


# Selected models, to choose from, based on the performance and size:
# YOLOv8:  'yolov8s', 'yolov8m'

# YOLOv10: 'yolov10s', 'yolov10m'

# YOLO12: 'yolo12s'

# Directory structure
MODELS_DIR = BASE_DIR / 'models' / MODEL_NAME
TMP_DIR = BASE_DIR / 'tmp' / MODEL_NAME

# Dataset Selection
# Option 1: Full dataset (~100k images) - for final optimization: "bdd100k_yolo"
# Option 2: Limited dataset (representative samples) - for quick tuning: "bdd100k_yolo_limited"
dataset_name = 'bdd100k_yolo_tuning'


YOLO_DATASET_ROOT = DATASET_BASE_DIR / dataset_name

# data.yaml path
DATA_YAML_PATH = YOLO_DATASET_ROOT / 'data.yaml'

# Verify dataset exists
if not DATA_YAML_PATH.exists():
    raise FileNotFoundError(
        f"Dataset not found: {DATA_YAML_PATH}\n"
        f"Please prepare the dataset first using process_bdd100k_to_yolo_dataset.py"
    )

# Update data.yaml path field for Colab compatibility
with open(DATA_YAML_PATH, 'r') as yaml_file:
    data_config = yaml.safe_load(yaml_file)

# Validate required keys in data.yaml
required_yaml_keys = ['nc', 'names', 'path']
missing_keys = [key for key in required_yaml_keys if key not in data_config]
if missing_keys:
    raise ValueError(f"Missing required keys in data.yaml: {missing_keys}")

# Update the 'path' field to use BASE_DIR
data_config['path'] = str(YOLO_DATASET_ROOT)

# Create a temporary data.yaml with corrected paths
temp_data_yaml = TMP_DIR / 'data.yaml'
TMP_DIR.mkdir(parents=True, exist_ok=True)
with open(temp_data_yaml, 'w') as yaml_output_file:
    yaml.dump(data_config, yaml_output_file, default_flow_style=False, sort_keys=False)

# Use the temporary data.yaml for training
DATA_YAML_PATH = temp_data_yaml

# Optimization Configuration
N_TRIALS = 40  # Number of optimization trials = 50–70 trials
TIMEOUT_HOURS = 24  # Maximum time for optimization (None for no limit)
N_STARTUP_TRIALS = 10  # Random exploration trials before optimization =10
EPOCHS_PER_TRIAL = 8  # Training epochs per trial = 50
BATCH_SIZE = 96  # Batch size for training
# for T4 GPU:
# 64 for 10n, 1 epoch 30 min
# 32 for 8m, 1 epoch 45 min

# for A100 GPU:
# 64 for 10m 1 epoch 11 min, 5 epochs completed in 0.797 hours.
# 96 for 8m , 1 epoch 10 min, 5 epochs completed in 0.866 hours.



# Weights & Biases (optional)
USE_WANDB = True  # Set to True to enable W&B logging
WANDB_PROJECT_TUNING = f"yolo-{YOLO_DATASET_ROOT.name}-tuning"

# ============================================================================
# RUN NAME CONFIGURATION - RESUME OR CREATE NEW
# ============================================================================
# To RESUME an existing run: Set RESUME_RUN_NAME to the run directory name
# To START NEW run: Leave RESUME_RUN_NAME as None or empty string
#
# Example to resume: RESUME_RUN_NAME = "yolov10n_tune_20251125_143022"
# ============================================================================

RESUME_RUN_NAME = "yolov8m_finetuned_1_tune_20251127_230340"  # Set to run name to resume, or None to create new run

if RESUME_RUN_NAME:
    # Resume existing run
    RUN_NAME_TUNING = RESUME_RUN_NAME
    print(f'\n🔄 RESUME MODE: Will attempt to resume run "{RESUME_RUN_NAME}"')
else:
    # Create new run with timestamp
    RUN_TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
    RUN_NAME_TUNING = f'{MODEL_NAME}_tune_{RUN_TIMESTAMP}'
    print(f'\n🆕 NEW RUN MODE: Creating new run "{RUN_NAME_TUNING}"')

RUN_NAME_TRAINING = f'{MODEL_NAME}_train_{RUN_TIMESTAMP if not RESUME_RUN_NAME else RESUME_RUN_NAME}'

# Create directories for tuning within tune_train folder
# All paths are absolute to ensure consistency across environments (local/Colab)
TUNE_TRAIN_BASE = BASE_DIR / 'tune_train'
TUNE_DIR = TUNE_TRAIN_BASE / 'tune' / RUN_NAME_TUNING
TUNE_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Keep RUN_DIR for backward compatibility (points to tuning)
RUN_DIR = TUNE_DIR
# Keep RUN_DIR for backward compatibility (points to tuning)
# Read dataset configuration
NUM_CLASSES = data_config['nc']
CLASS_NAMES = {i: name for i, name in enumerate(data_config['names'])}
CLASS_NAME_TO_ID = {name: i for i, name in enumerate(data_config['names'])}

print('=' * 80)
print('CONFIGURATION SUMMARY')
print('=' * 80)
print(f'Environment: {"Google Colab" if "COLAB_GPU" in os.environ or os.path.exists("/content") else "Local"}')
print(f'Base Directory: {BASE_DIR}')
print(f'Model: {MODEL_NAME}')
print(f'Dataset: {YOLO_DATASET_ROOT.name}')
print(f'Data YAML: {DATA_YAML_PATH}')
print(f'  Dataset path in YAML: {data_config["path"]}')
print(f'Classes: {NUM_CLASSES}')
print(f'Class Names: {CLASS_NAMES}')
print(f'Device: {device}')
print(f'Optimization Trials: {N_TRIALS}')
print(f'Epochs per Trial: {EPOCHS_PER_TRIAL}')
print(f'Batch Size: {BATCH_SIZE}')
print(f'Timeout: {TIMEOUT_HOURS} hours' if TIMEOUT_HOURS else 'No timeout')
print(f'Tuning Directory: {TUNE_DIR}')
if USE_WANDB:
    print(f'W&B Logging: Enabled')
    print(f'  Tuning Project: {WANDB_PROJECT_TUNING}')
else:
    print(f'W&B Logging: Disabled')
print('=' * 80)


🔄 RESUME MODE: Will attempt to resume run "yolov8m_finetuned_1_tune_20251127_230340"
CONFIGURATION SUMMARY
Environment: Google Colab
Base Directory: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo
Model: yolov8m_finetuned_1
Dataset: bdd100k_yolo_tuning
Data YAML: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tmp/yolov8m_finetuned_1/data.yaml
  Dataset path in YAML: /computer_vision_yolo/bdd100k_yolo_tuning
Classes: 10
Class Names: {0: 'person', 1: 'rider', 2: 'car', 3: 'truck', 4: 'bus', 5: 'train', 6: 'motor', 7: 'bike', 8: 'traffic light', 9: 'traffic sign'}
Device: cuda
Optimization Trials: 40
Epochs per Trial: 8
Batch Size: 96
Timeout: 24 hours
Tuning Directory: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340
W&B Logging: Enabled
  Tuning Project: yolo-bdd100k_yolo_tuning-tuning


## 4. Load Base YOLO Model

In [8]:
# Load YOLO model with automatic download
model_path = MODELS_DIR / f'{MODEL_NAME}.pt'

if not model_path.exists():
    print(f'Model not found at {model_path}')
    print(f'Downloading {MODEL_NAME} ...')

    try:
        # Download model - ensure .pt extension for ultralytics
        # Ultralytics expects model names with .pt extension for download
        if not MODEL_NAME.endswith('.pt'):
            model_name_for_download = MODEL_NAME + '.pt'
        else:
            model_name_for_download = MODEL_NAME

        print(f'  Requesting model: {model_name_for_download}')
        model = YOLO(model_name_for_download)

        # Create models directory
        MODELS_DIR.mkdir(parents=True, exist_ok=True)

        # Save model to our directory using export/save
        try:
            # Try to save using the model's save method
            if hasattr(model, 'save'):
                model.save(str(model_path))
                print(f'✓ Model downloaded and saved to {model_path}')
                print(f'  Size: {model_path.stat().st_size / (1024*1024):.1f} MB')
            else:
                # Fallback: copy from cache
                cache_patterns = [
                    str(Path.home() / '.cache' / 'ultralytics' / '**' / f'{MODEL_NAME}.pt'),
                    str(Path.home() / '.config' / 'Ultralytics' / '**' / f'{MODEL_NAME}.pt'),
                ]

                model_found = False
                for pattern in cache_patterns:
                    cache_paths = glob.glob(pattern, recursive=True)
                    if cache_paths:
                        shutil.copy(cache_paths[0], model_path)
                        print(f'✓ Model downloaded and saved to {model_path}')
                        print(f'  Size: {model_path.stat().st_size / (1024*1024):.1f} MB')
                        model_found = True
                        break

                if not model_found:
                    print(f'✓ Model loaded from ultralytics cache')
                    print(f'  Note: Model is in cache, not copied to {model_path}')
                    print(f'  This is normal and the model will work correctly')
        except Exception as save_error:
            print(f'⚠️  Could not save model to custom location: {save_error}')
            print(f'✓ Model loaded successfully from ultralytics cache')

    except Exception as download_error:
        print(f'\n❌ Error downloading model: {download_error}')
        raise
else:
    model = YOLO(str(model_path))
    print(f'✓ Model loaded from {model_path}')

# Get model information
model_info_dict = {}
model_info_result = model.info()
model_info_keys = ["layers", "params", "size(MB)", "FLOPs(G)"]

for info_key, info_value in zip(model_info_keys, model_info_result):
    model_info_dict[info_key] = info_value

model_params = model_info_dict.get("params", 0)
model_size_mb = model_info_dict.get("size(MB)", 0)
flops_gflops = model_info_dict.get("FLOPs(G)", 0)


print(f'\n📊 Model Information:')
print(f'  Model: {MODEL_NAME}')
print(f'  Classes in model: {len(model.names)}')
print(f'  Task: {model.task}')
print(f'  Parameters: {model_params / 1e6:.1f}M')
print(f'  Model Size: {model_size_mb:.1f} MB')
print(f'  FLOPs (640x640): {flops_gflops:.2f} GFLOPs')

✓ Model loaded from /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/models/yolov8m_finetuned_1/yolov8m_finetuned_1.pt
Model summary: 169 layers, 25,862,110 parameters, 0 gradients, 79.1 GFLOPs

📊 Model Information:
  Model: yolov8m_finetuned_1
  Classes in model: 10
  Task: detect
  Parameters: 25.9M
  Model Size: 0.0 MB
  FLOPs (640x640): 79.09 GFLOPs


## 5. Verify Dataset Structure

In [9]:
# ============================================================================
# VERIFY DATASET STRUCTURE
# ============================================================================

print('Verifying YOLO dataset structure...')
print(f'\n📁 Dataset Root: {YOLO_DATASET_ROOT}')

# Check all splits using constants
dataset_stats = {}
for split in [DatasetSplit.TRAIN, DatasetSplit.VAL, DatasetSplit.TEST]:
    images_dir = YOLO_DATASET_ROOT / 'images' / split
    labels_dir = YOLO_DATASET_ROOT / 'labels' / split

    if images_dir.exists() and labels_dir.exists():
        num_images = len(list(images_dir.glob('*.jpg'))) + len(list(images_dir.glob('*.png')))
        num_labels = len(list(labels_dir.glob('*.txt')))
        dataset_stats[split] = {'images': num_images, 'labels': num_labels}
        print(f'  ✓ {split:5s}: {num_images:6d} images, {num_labels:6d} labels')
    else:
        print(f'  ⚠️  {split:5s}: Directory not found')
        dataset_stats[split] = {'images': 0, 'labels': 0}

print(f'\n📄 Configuration: {DATA_YAML_PATH}')
print(f'  Classes: {NUM_CLASSES}')
print(f'  Names: {CLASS_NAMES}')

total_images = sum(stats['images'] for stats in dataset_stats.values())
print(f'\n✓ Dataset verified: {total_images:,} total images')
print('✓ Ready for hyperparameter optimization')

Verifying YOLO dataset structure...

📁 Dataset Root: /computer_vision_yolo/bdd100k_yolo_tuning
  ✓ train:  16391 images,  16391 labels
  ✓ val  :  10000 images,  10000 labels
  ⚠️  test : Directory not found

📄 Configuration: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tmp/yolov8m_finetuned_1/data.yaml
  Classes: 10
  Names: {0: 'person', 1: 'rider', 2: 'car', 3: 'truck', 4: 'bus', 5: 'train', 6: 'motor', 7: 'bike', 8: 'traffic light', 9: 'traffic sign'}

✓ Dataset verified: 26,391 total images
✓ Ready for hyperparameter optimization


## 6. Define Hyperparameter Search Space

In [10]:
# ============================================================================
# DEFINE FOCUSED HYPERPARAMETER SEARCH SPACE
# ============================================================================

def define_hyperparameters(trial):
    """
    Focused hyperparameter search for YOLO - only critical high-impact parameters.

    Args:
        trial: Optuna trial object for sampling hyperparameters

    Returns:
        dict: Dictionary of hyperparameters for YOLO training

    Tuning Strategy:
    - Focus ONLY on parameters with proven high impact on performance
    - Use YOLO defaults for well-calibrated parameters (HSV, loss weights)
    - Reduces search space for faster convergence and better results

    Critical Parameters Tuned:
    1. Image size (imgsz): 640, 800, 1024
    2. Batch size: Dynamically adjusted based on image size (96 for 640, 64 for 800+)
    3. Optimizer choice (SGD/Adam/AdamW)
    4. Initial learning rate (lr0): 1e-4 to 5e-3
    5. Momentum/beta1: 0.85 to 0.97
    6. Weight decay (regularization): 1e-5 to 1e-3
    7. Warmup epochs: 0 to 3
    8. Warmup momentum: 0.5 to 0.95
    9. Warmup bias learning rate: 0.0 to 0.1
    10. Mosaic augmentation strength: 0.5 to 1.0
    11. Mixup augmentation strength: 0.0 to 0.2
    """

    if trial is None:
        raise ValueError("Trial object cannot be None")

    # ---------------------------
    # 1) Image Size
    # ---------------------------
    # Test different image sizes to find optimal accuracy/speed tradeoff
    image_size = trial.suggest_categorical('imgsz', [640, 768])

    # ---------------------------
    # 2) Batch Size (Dynamic based on image size)
    # ---------------------------
    # Larger images require more memory, so reduce batch size accordingly
    if image_size == 640:
        batch_size = 96  # Standard batch size for 640x640
    else:  # 768
        batch_size = 64  # Reduced batch size for larger images

    # ---------------------------
    # 3) Optimizer + Learning Rate
    # ---------------------------
    optimizer_choice = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW'])
    lr0 = trial.suggest_float('lr0', 1e-4, 5e-3, log=True)

    # ---------------------------
    # 4) Regularization
    # ---------------------------
    momentum = trial.suggest_float('momentum', 0.85, 0.97)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)

    # ---------------------------
    # 5) Warmup Configuration
    # ---------------------------
    warmup_epochs = trial.suggest_int('warmup_epochs', 0, 3)
    warmup_momentum = trial.suggest_float('warmup_momentum', 0.5, 0.95)
    warmup_bias_lr = trial.suggest_float('warmup_bias_lr', 0.0, 0.1)

    # ---------------------------
    # 6) Key Augmentation
    # ---------------------------
    # Mosaic and mixup have the highest impact on performance
    mosaic = trial.suggest_float('mosaic', 0.5, 1.0)
    mixup = trial.suggest_float('mixup', 0.0, 0.2)

    # ---------------------------
    # 7) Compile parameters
    # ---------------------------
    hyperparams = {
        # ===== TUNED PARAMETERS (Critical for performance) =====
        'imgsz': image_size,
        'batch': batch_size,
        'optimizer': optimizer_choice,
        'lr0': lr0,
        'momentum': momentum,
        'weight_decay': weight_decay,
        'warmup_epochs': warmup_epochs,
        'warmup_momentum': warmup_momentum,
        'warmup_bias_lr': warmup_bias_lr,
        'mosaic': mosaic,
        'mixup': mixup,

        # ===== DEFAULT PARAMETERS (YOLO defaults work well) =====
        # Learning rate decay: default 0.01 is well-calibrated
        # HSV augmentation: defaults (0.015, 0.7, 0.4) are optimal for most cases
        # Spatial augmentation: defaults for scale/translate work well
        # Loss weights: YOLO defaults (7.5, 0.5, 1.5) are well-balanced

        # ===== FIXED PARAMETERS =====
        'epochs': EPOCHS_PER_TRIAL,
        'device': device,
        'val': True,
        'patience': ModelConfig.DEFAULT_PATIENCE,
        'save': True,
        'plots': True,
        'cache': False,
        'workers': ModelConfig.DEFAULT_WORKERS,
        'close_mosaic': ModelConfig.CLOSE_MOSAIC_EPOCHS,
        'verbose': True,
    }

    return hyperparams


print('✓ Hyperparameter search space defined')
print('\n📊 Focused Search Space Summary:')
print('  Strategy: Tune ONLY critical high-impact parameters')
print('  🎯 Tuned Parameters (11):')
print('    - Image Size (imgsz): 640, 800, 1024')
print('    - Batch Size: Dynamic (96 for 640, 64 for 800+)')
print('    - Optimizer: SGD, Adam, AdamW')
print('    - Learning Rate (lr0): 1e-4 to 5e-3')
print('    - Momentum: 0.85 to 0.97')
print('    - Weight Decay: 1e-5 to 1e-3')
print('    - Warmup Epochs: 0 to 3')
print('    - Warmup Momentum: 0.5 to 0.95')
print('    - Warmup Bias LR: 0.0 to 0.1')
print('    - Mosaic: 0.5 to 1.0')
print('    - Mixup: 0.0 to 0.2')
print('  ⚙️  Fixed Parameters:')
print(f'    - Epochs: {EPOCHS_PER_TRIAL}')
print(f'    - Device: {device}')
print('  📌 Using YOLO defaults for: HSV augmentation, spatial transforms, loss weights')

✓ Hyperparameter search space defined

📊 Focused Search Space Summary:
  Strategy: Tune ONLY critical high-impact parameters
  🎯 Tuned Parameters (11):
    - Image Size (imgsz): 640, 800, 1024
    - Batch Size: Dynamic (96 for 640, 64 for 800+)
    - Optimizer: SGD, Adam, AdamW
    - Learning Rate (lr0): 1e-4 to 5e-3
    - Momentum: 0.85 to 0.97
    - Weight Decay: 1e-5 to 1e-3
    - Warmup Epochs: 0 to 3
    - Warmup Momentum: 0.5 to 0.95
    - Warmup Bias LR: 0.0 to 0.1
    - Mosaic: 0.5 to 1.0
    - Mixup: 0.0 to 0.2
  ⚙️  Fixed Parameters:
    - Epochs: 8
    - Device: cuda
  📌 Using YOLO defaults for: HSV augmentation, spatial transforms, loss weights


## 7. Define Objective Function

In [11]:
# DEFINE OBJECTIVE FUNCTION FOR OPTUNA
# ============================================================================

def objective(trial):
    """Objective function for Optuna hyperparameter optimization.

    Steps:
    1. Sample hyperparameters for the current trial
    2. Train a YOLO model with those hyperparameters
    3. Evaluate the model on the validation set
    4. Return validation mAP@0.5 (to maximize)
    """
    # Get hyperparameters for this trial
    hyperparameters = define_hyperparameters(trial)

    # Create trial-specific directory (absolute path under BASE_DIR)
    trial_dir = TUNE_DIR / f"trial_{trial.number:03d}"
    trial_dir.mkdir(exist_ok=True, parents=True)

    # Initialize W&B if enabled
    wandb_run = None
    if USE_WANDB:
        try:
            os.environ['WANDB_DIR'] = str(trial_dir)
            wandb_run = wandb.init(
                project=WANDB_PROJECT_TUNING,
                name=f'{MODEL_NAME}_trial_{trial.number:03d}',
                config=hyperparameters,
                dir=str(trial_dir),
                reinit=True
            )
        except Exception as wandb_error:
            print(f'⚠️  W&B initialization failed: {wandb_error}')
            wandb_run = None

    # Print trial information
    print(f"\n{'=' * 80}")
    print(f"TRIAL {trial.number}/{N_TRIALS}")
    print(f"{'=' * 80}")
    print(f"🎯 Tuned Parameters:")
    print(f"  Image Size: {hyperparameters['imgsz']}")
    print(f"  Batch Size: {hyperparameters['batch']} (auto-adjusted for image size)")
    print(f"  Optimizer: {hyperparameters['optimizer']}")
    print(f"  Learning Rate: {hyperparameters['lr0']:.6f}")
    print(f"  Momentum: {hyperparameters['momentum']:.4f}")
    print(f"  Weight Decay: {hyperparameters['weight_decay']:.6f}")
    print(f"  Warmup: epochs={hyperparameters['warmup_epochs']}, momentum={hyperparameters['warmup_momentum']:.2f}, bias_lr={hyperparameters['warmup_bias_lr']:.3f}")
    print(f"  Mosaic: {hyperparameters['mosaic']:.2f}")
    print(f"  Mixup: {hyperparameters['mixup']:.2f}")
    print(f"✓ Using YOLO defaults for: HSV, spatial aug, loss weights, lrf")
    print(f"{'=' * 80}")

    trial_model = None
    map50 = 0.001  # Default penalty for failed trials

    try:
        # Load fresh model for this trial
        trial_model = YOLO(str(model_path))

        # Train model with hyperparameters (W&B integration via wandb.init)
        trial_run_name = f"{MODEL_NAME}_trial_{trial.number:03d}"
        train_results = trial_model.train(
            data=str(DATA_YAML_PATH),
            project=str(trial_dir),
            name=trial_run_name,
            exist_ok=True,
            **hyperparameters,
        )

        # Validate model
        validation_results = trial_model.val(
            data=str(DATA_YAML_PATH),
            split="val",
            project=str(trial_dir),
            name="val",
            verbose=False,
        )

        # Extract metrics
        map50 = float(validation_results.box.map50)
        map50_95 = float(validation_results.box.map)
        precision = float(validation_results.box.mp)
        recall = float(validation_results.box.mr)

        # Save training metrics if available
        train_metrics = {}
        if hasattr(train_results, 'results_dict'):
            train_metrics = {key: float(value) if isinstance(value, (int,float,np.floating,np.integer)) else value
                             for key,value in train_results.results_dict.items()
                             if key not in ['fitness']}

        # Save trial results JSON
        trial_results = {
            "trial_number": trial.number,
            "model_name": MODEL_NAME,
            "dataset": YOLO_DATASET_ROOT.name,
            "trial_directory": str(trial_dir),
            "hyperparameters": {k: float(v) if isinstance(v,(np.floating,np.integer)) else v for k,v in hyperparameters.items()},
            "validation_metrics": {"map50": map50, "map50_95": map50_95, "precision": precision, "recall": recall},
            "training_metrics": train_metrics,
            "training_config": {
                "epochs": EPOCHS_PER_TRIAL,
                "batch_size": hyperparameters['batch'],
                "image_size": hyperparameters['imgsz'],
                "device": device,
            },
            "timestamp": datetime.now().isoformat(),
            "status": "completed"
        }

        trial_results_path = trial_dir / "trial_results.json"
        with open(trial_results_path, 'w', encoding='utf-8') as f:
            json.dump(trial_results, f, indent=2)

        print(f'\n✅ Trial {trial.number} Completed')
        print(f'  mAP@0.5: {map50:.4f}')
        print(f'  mAP@0.5:0.95: {map50_95:.4f}')
        print(f'  Precision: {precision:.4f}')
        print(f'  Recall: {recall:.4f}')

    except Exception as error:
        print(f'\n❌ Trial {trial.number} Failed: {error}')

        # Save error information
        trial_results = {
            "trial_number": trial.number,
            "model_name": MODEL_NAME,
            "dataset": YOLO_DATASET_ROOT.name,
            "trial_directory": str(trial_dir),
            "hyperparameters": {k: float(v) if isinstance(v,(np.floating,np.integer)) else v for k,v in hyperparameters.items()},
            "error": str(error),
            "timestamp": datetime.now().isoformat(),
            "status": "failed"
        }

        trial_results_path = trial_dir / "trial_results.json"
        with open(trial_results_path, 'w', encoding='utf-8') as f:
            json.dump(trial_results, f, indent=2)

        # Return small penalty value instead of raising exception
        map50 = 0.001

    finally:
        # Clean up
        if wandb_run is not None:
            wandb_run.finish()

        # Clean up trial model
        if trial_model is not None:
            del trial_model

        # Force garbage collection
        gc.collect()
        if device == 'cuda':
            torch.cuda.empty_cache()
            print("🧹 CUDA cache cleared")

    return map50


print('✓ Objective function defined')
print('  Returns: mAP@0.5 (validation set)')
print('  Goal: Maximize validation performance')

✓ Objective function defined
  Returns: mAP@0.5 (validation set)
  Goal: Maximize validation performance


## 8. Run Hyperparameter Optimization

In [12]:
# RUN HYPERPARAMETER OPTIMIZATION WITH OPTUNA
# ============================================================================

print('\n' + '=' * 80)
print('STARTING HYPERPARAMETER OPTIMIZATION')
print('=' * 80)
print(f'Model: {MODEL_NAME}')
print(f'Dataset: {YOLO_DATASET_ROOT.name}')
print(f'Number of Trials: {N_TRIALS}')
print(f'Epochs per Trial: {EPOCHS_PER_TRIAL}')
print(f'Timeout: {TIMEOUT_HOURS} hours' if TIMEOUT_HOURS else 'No timeout')
print(f'Device: {device}')
print('=' * 80)

# Check if resuming from previous run
study_pkl_path = TUNE_DIR / 'optuna_study.pkl'
checkpoint_log_path = TUNE_DIR / 'checkpoint_log.json'
is_resuming = study_pkl_path.exists()

if is_resuming:
    # Load existing study
    print('\n' + '=' * 80)
    print('🔄 RESUMING PREVIOUS OPTIMIZATION')
    print('=' * 80)

    with open(study_pkl_path, 'rb') as f:
        study = pickle.load(f)

    # Load checkpoint log
    checkpoint_data = []
    if checkpoint_log_path.exists():
        with open(checkpoint_log_path, 'r', encoding='utf-8') as f:
            checkpoint_data = json.load(f)

    # Display resume information
    completed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])
    pruned_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])
    failed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])
    total_previous_trials = len(study.trials)

    print(f'\n📊 Previous Run Summary:')
    print(f'  Completed Trials: {completed_trials}')
    print(f'  Pruned Trials: {pruned_trials}')
    print(f'  Failed Trials: {failed_trials}')
    print(f'  Total Previous Trials: {total_previous_trials}')

    if completed_trials > 0:
        best_trial = study.best_trial
        print(f'\n🏆 Best Result So Far:')
        print(f'  Trial: {best_trial.number}')
        print(f'  mAP@0.5: {best_trial.value:.4f}')

        # Show top 3 completed trials
        completed_trial_list = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
        sorted_trials = sorted(completed_trial_list, key=lambda t: t.value, reverse=True)
        top_3_trials = sorted_trials[:3]

        print(f'\n📈 Top 3 Trials:')
        for idx, trial in enumerate(top_3_trials, 1):
            print(f'  {idx}. Trial {trial.number}: mAP@0.5 = {trial.value:.4f}')

    # Show last checkpoint info
    if checkpoint_data:
        last_checkpoint = checkpoint_data[-1]
        print(f'\n🕐 Last Checkpoint:')
        print(f'  Timestamp: {last_checkpoint["timestamp"]}')
        print(f'  Last Trial: {last_checkpoint["trial_number"]}')
        print(f'  Current Best mAP: {last_checkpoint["best_map"]:.4f}')

    remaining_trials = N_TRIALS - total_previous_trials
    print(f'\n➡️  Continuing optimization: {remaining_trials} trials remaining (of {N_TRIALS} total)')
    print('=' * 80)

else:
    # Create new Optuna study
    print('\n🆕 Creating new optimization study')

    study = optuna.create_study(
        study_name=f'{MODEL_NAME}_optuna_{RUN_TIMESTAMP}',
        direction='maximize',  # Maximize mAP@0.5
        sampler=optuna.samplers.TPESampler(
            seed=42,
            n_startup_trials=N_STARTUP_TRIALS,  # Random trials before optimization
            multivariate=True,  # Consider parameter interactions
            group=True  # Group related parameters
        ),
        pruner=optuna.pruners.MedianPruner(
            n_startup_trials=N_STARTUP_TRIALS,
            n_warmup_steps=15,  # Wait before pruning
            interval_steps=5  # Check every 5 steps
        )
    )

    # Initialize checkpoint log
    checkpoint_data = []

# Run optimization
start_time = datetime.now()
print(f'\n🚀 Optimization started at {start_time.strftime("%Y-%m-%d %H:%M:%S")}')

# Define checkpoint callback
def checkpoint_callback(study, trial):
    """Save checkpoint after each trial completion"""
    print(f'\n✓ Completed {len(study.trials)}/{N_TRIALS} trials')

    # Update checkpoint log
    checkpoint_entry = {
        'trial_number': trial.number,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'trial_state': trial.state.name,
        'best_map': study.best_value if len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]) > 0 else 0.0,
        'completed_trials': len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]),
        'total_trials': len(study.trials)
    }
    checkpoint_data.append(checkpoint_entry)

    # Save checkpoint log
    with open(checkpoint_log_path, 'w', encoding='utf-8') as f:
        json.dump(checkpoint_data, f, indent=2)

    # Save study object
    with open(study_pkl_path, 'wb') as f:
        pickle.dump(study, f)

    # Force garbage collection
    gc.collect()

try:
    study.optimize(
        objective,
        n_trials=N_TRIALS,
        timeout=TIMEOUT_HOURS * 3600 if TIMEOUT_HOURS else None,
        show_progress_bar=True,
        callbacks=[checkpoint_callback]
    )
except KeyboardInterrupt:
    print('\n⚠️  Optimization interrupted by user')
    print(f'💾 Progress saved to: {TUNE_DIR}')
    print(f'   - Study checkpoint: {study_pkl_path.name}')
    print(f'   - Checkpoint log: {checkpoint_log_path.name}')
    print(f'\n🔄 To resume: Simply re-run this notebook')
except Exception as e:
    print(f'\n❌ Optimization failed: {e}')
    import traceback
    traceback.print_exc()

end_time = datetime.now()
duration = end_time - start_time

print('\n' + '=' * 80)
print('OPTIMIZATION COMPLETED')
print('=' * 80)
print(f'Started: {start_time.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Ended: {end_time.strftime("%Y-%m-%d %H:%M:%S")}')
print(f'Duration: {duration}')
print(f'Total Trials: {len(study.trials)}')
print(f'Completed Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}')
print(f'Pruned Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}')
print(f'Failed Trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL])}')
print(f'\nBest Trial: {study.best_trial.number}')
print(f'Best mAP@0.5: {study.best_value:.4f}')
print('=' * 80)


STARTING HYPERPARAMETER OPTIMIZATION
Model: yolov8m_finetuned_1
Dataset: bdd100k_yolo_tuning
Number of Trials: 40
Epochs per Trial: 8
Timeout: 24 hours
Device: cuda

🔄 RESUMING PREVIOUS OPTIMIZATION

📊 Previous Run Summary:
  Completed Trials: 35
  Pruned Trials: 0
  Failed Trials: 0
  Total Previous Trials: 35

🏆 Best Result So Far:
  Trial: 0
  mAP@0.5: 0.5769

📈 Top 3 Trials:
  1. Trial 0: mAP@0.5 = 0.5769
  2. Trial 22: mAP@0.5 = 0.5767
  3. Trial 11: mAP@0.5 = 0.5765

🕐 Last Checkpoint:
  Timestamp: 2025-11-28 17:18:28
  Last Trial: 34
  Current Best mAP: 0.5769

➡️  Continuing optimization: 5 trials remaining (of 40 total)

🚀 Optimization started at 2025-11-28 17:40:46


  0%|          | 0/40 [00:00<?, ?it/s]

wandb: Currently logged in as: m3mahdy (m3mahdy-king-saud-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.



TRIAL 35/40
🎯 Tuned Parameters:
  Image Size: 768
  Batch Size: 64 (auto-adjusted for image size)
  Optimizer: SGD
  Learning Rate: 0.000161
  Momentum: 0.9099
  Weight Decay: 0.000428
  Warmup: epochs=1, momentum=0.51, bias_lr=0.019
  Mosaic: 0.82
  Mixup: 0.02
✓ Using YOLO defaults for: HSV, spatial aug, loss weights, lrf
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tmp/yolov8m_finetuned_1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=8, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freez

🧹 CUDA cache cleared
[I 2025-11-28 18:09:12,341] Trial 35 finished with value: 0.5770742349894156 and parameters: {'imgsz': 768, 'optimizer': 'SGD', 'lr0': 0.0001614596240725037, 'momentum': 0.9098500376932075, 'weight_decay': 0.00042786651935909105, 'warmup_epochs': 1, 'warmup_momentum': 0.5149307497845836, 'warmup_bias_lr': 0.018938388657529233, 'mosaic': 0.8205379408033115, 'mixup': 0.023789168654792303}. Best is trial 35 with value: 0.5770742349894156.

✓ Completed 36/40 trials


...

🧹 CUDA cache cleared
[I 2025-11-28 19:47:36,439] Trial 39 finished with value: 0.5548905335656567 and parameters: {'imgsz': 640, 'optimizer': 'SGD', 'lr0': 0.00025969974443728553, 'momentum': 0.9010564532726025, 'weight_decay': 0.00015395200555927614, 'warmup_epochs': 0, 'warmup_momentum': 0.5678979552619839, 'warmup_bias_lr': 0.008529487882382562, 'mosaic': 0.7499675740997609, 'mixup': 0.02807877625623379}. Best is trial 35 with value: 0.5770742349894156.

✓ Completed 40/40 trials



TRIAL 40/40
🎯 Tuned Parameters:
  Image Size: 640
  Batch Size: 96 (auto-adjusted for image size)
  Optimizer: SGD
  Learning Rate: 0.000349
  Momentum: 0.8559
  Weight Decay: 0.000991
  Warmup: epochs=1, momentum=0.87, bias_lr=0.008
  Mosaic: 0.97
  Mixup: 0.20
✓ Using YOLO defaults for: HSV, spatial aug, loss weights, lrf
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=96, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tmp/yolov8m_finetuned_1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=8, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freez

🧹 CUDA cache cleared
[I 2025-11-28 20:09:14,233] Trial 40 finished with value: 0.5561320329377647 and parameters: {'imgsz': 640, 'optimizer': 'SGD', 'lr0': 0.0003489202891839771, 'momentum': 0.8559053049915213, 'weight_decay': 0.000991194711322742, 'warmup_epochs': 1, 'warmup_momentum': 0.8714204009107174, 'warmup_bias_lr': 0.0076481178071809, 'mosaic': 0.9650376225502465, 'mixup': 0.19702415968347725}. Best is trial 35 with value: 0.5770742349894156.

✓ Completed 41/40 trials



TRIAL 41/40
🎯 Tuned Parameters:
  Image Size: 768
  Batch Size: 64 (auto-adjusted for image size)
  Optimizer: SGD
  Learning Rate: 0.000236
  Momentum: 0.9335
  Weight Decay: 0.000335
  Warmup: epochs=1, momentum=0.54, bias_lr=0.023
  Mosaic: 0.90
  Mixup: 0.01
✓ Using YOLO defaults for: HSV, spatial aug, loss weights, lrf
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tmp/yolov8m_finetuned_1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=8, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freez

🧹 CUDA cache cleared
[W 2025-11-28 20:18:13,970] Trial 41 failed with parameters: {'imgsz': 768, 'optimizer': 'SGD', 'lr0': 0.00023610953279924895, 'momentum': 0.9334630232585438, 'weight_decay': 0.0003346298991138973, 'warmup_epochs': 1, 'warmup_momentum': 0.5409541831782676, 'warmup_bias_lr': 0.02284296681450065, 'mosaic': 0.8966792095706233, 'mixup': 0.005637501635348943} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-848837866.py", line 62, in objective
    train_results = trial_model.train(
                    ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/model.py", line 773, in train
    self.trainer.train()
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/engine/trainer.py", line 243, in train
   

## 9. Save All Trials Summary

In [13]:
# SAVE CONSOLIDATED SUMMARY OF ALL TRIALS
# ============================================================================

print('\n' + '=' * 80)
print('SAVING CONSOLIDATED TRIAL SUMMARY')
print('=' * 80)

# Collect all trial results dynamically from study
all_trials_data = []

for trial in study.trials:
    trial_dir = TUNE_DIR / f"trial_{trial.number:03d}"
    results_file = trial_dir / "trial_results.json"

    if results_file.exists():
        try:
            with open(results_file, 'r') as f:
                trial_data = json.load(f)
                all_trials_data.append(trial_data)
        except Exception as e:
            print(f"⚠️  Could not read trial {trial.number} results: {e}")
    else:
        print(f"⚠️  No results file found for trial {trial.number}")

# Create comprehensive summary
optimization_summary = {
    "model_name": MODEL_NAME,
    "dataset": YOLO_DATASET_ROOT.name,
    "optimization_config": {
        "n_trials": N_TRIALS,
        "epochs_per_trial": EPOCHS_PER_TRIAL,
        "batch_size": BATCH_SIZE,
        "timeout_hours": TIMEOUT_HOURS,
        "n_startup_trials": N_STARTUP_TRIALS,
    },
    "optimization_results": {
        "start_time": start_time.isoformat(),
        "end_time": end_time.isoformat(),
        "duration_seconds": duration.total_seconds(),
        "total_trials": len(study.trials),
        "completed_trials": len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]),
        "pruned_trials": len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]),
        "failed_trials": len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL]),
        "best_trial_number": study.best_trial.number,
        "best_map50": study.best_value,
    },
    "best_hyperparameters": study.best_params,
    "all_trials": all_trials_data,
    "timestamp": datetime.now().isoformat(),
}

# Save consolidated summary as JSON
summary_path = TUNE_DIR / f"{MODEL_NAME}_all_trials_summary.json"
with open(summary_path, 'w') as f:
    json.dump(optimization_summary, f, indent=2)

print(f'✓ Consolidated JSON summary saved: {summary_path}')
print(f'  Total trials saved: {len(all_trials_data)}')

# Create CSV summary for easy analysis
csv_data = []
for trial_data in all_trials_data:
    row = {
        'trial_number': trial_data.get('trial_number'),
        'status': trial_data.get('status'),
        'map50': trial_data.get('validation_metrics', {}).get('map50'),
        'map50_95': trial_data.get('validation_metrics', {}).get('map50_95'),
        'precision': trial_data.get('validation_metrics', {}).get('precision'),
        'recall': trial_data.get('validation_metrics', {}).get('recall'),
        'error_type': trial_data.get('error_type', '')  # Include error type if failed
    }
    # Add hyperparameters
    for key, value in trial_data.get('hyperparameters', {}).items():
        row[f'hp_{key}'] = value
    # Flag best trial
    row['best_trial'] = trial_data.get('trial_number') == study.best_trial.number
    csv_data.append(row)

df_trials = pd.DataFrame(csv_data)

# Sort CSV by mAP@0.5 descending (best first)
df_trials.sort_values(by='map50', ascending=False, inplace=True)

# Save CSV
csv_path = TUNE_DIR / f"{MODEL_NAME}_all_trials_summary.csv"
df_trials.to_csv(csv_path, index=False)

print(f'✓ CSV summary saved: {csv_path}')
print(f'  Columns: {len(df_trials.columns)}, Rows: {len(df_trials)}')
print('=' * 80)

# Display summary statistics
if len(df_trials) > 0:
    print('\n📊 Trial Summary Statistics:')
    print(f'  Completed Trials: {len(df_trials[df_trials["status"] == "completed"])}')
    print(f'  Failed Trials: {len(df_trials[df_trials["status"] == "failed"])}')

    completed_trials = df_trials[df_trials['status'] == 'completed']
    if len(completed_trials) > 0:
        best_trial_row = completed_trials.loc[completed_trials["map50"].idxmax()]
        print(f'\n  mAP@0.5 Statistics:')
        print(f'    Best: {best_trial_row["map50"]:.4f} (Trial {best_trial_row["trial_number"]})')
        print(f'    Worst: {completed_trials["map50"].min():.4f}')
        print(f'    Mean: {completed_trials["map50"].mean():.4f}')
        print(f'    Std: {completed_trials["map50"].std():.4f}')
        print(f'    Median: {completed_trials["map50"].median():.4f}')
print('=' * 80)


SAVING CONSOLIDATED TRIAL SUMMARY
⚠️  No results file found for trial 41
✓ Consolidated JSON summary saved: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/yolov8m_finetuned_1_all_trials_summary.json
  Total trials saved: 41
✓ CSV summary saved: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/yolov8m_finetuned_1_all_trials_summary.csv
  Columns: 29, Rows: 41

📊 Trial Summary Statistics:
  Completed Trials: 41
  Failed Trials: 0

  mAP@0.5 Statistics:
    Best: 0.5771 (Trial 35)
    Worst: 0.5258
    Mean: 0.5647
    Std: 0.0126
    Median: 0.5673


## 10. Save Best Hyperparameters

In [14]:
# SAVE BEST HYPERPARAMETERS
# ============================================================================

print('\n' + '=' * 80)
print('SAVING BEST HYPERPARAMETERS')
print('=' * 80)

# Extract best parameters from study
best_params = study.best_params
best_trial = study.best_trial

print(f'\n🏆 Best Trial: {best_trial.number}')
print(f'   Best mAP@0.5: {study.best_value:.4f}')
print('\n📋 Best Hyperparameters:')
for param_name, param_value in best_params.items():
    print(f'   {param_name}: {param_value}')

# Save best hyperparameters to JSON
best_params_json = TUNE_DIR / 'best_hyperparameters.json'
with open(best_params_json, 'w') as f:
    json.dump({
        'model': MODEL_NAME,
        'dataset_root': str(YOLO_DATASET_ROOT),
        'data_yaml_path': str(DATA_YAML_PATH),
        'optimization_results': {
            'best_trial': study.best_trial.number,
            'best_map50': study.best_value,
            'total_trials': len(study.trials),
            'optimization_duration': str(duration),
        },
        'hyperparameters': best_params,
        'timestamp': datetime.now().isoformat(),
        'notes': 'Use these hyperparameters for training. Add epochs, batch, imgsz, device, and other training settings.'
    }, f, indent=2)

print(f'\n✓ Best hyperparameters saved to: {best_params_json}')

# Save to YAML format (ready for YOLO training)
best_params_yaml = TUNE_DIR / 'best_hyperparameters.yaml'
with open(best_params_yaml, 'w') as f:
    yaml.dump(best_params, f, default_flow_style=False, sort_keys=False)

print(f'✓ Best hyperparameters saved to: {best_params_yaml}')

print('\n📋 Best Hyperparameters Summary:')
print(f'  Optimizer: {best_params.get("optimizer", "N/A")}')
print(f'  Learning Rate: {best_params.get("lr0", 0):.6f}')
print(f'  Momentum: {best_params.get("momentum", 0):.4f}')
print(f'  Weight Decay: {best_params.get("weight_decay", 0):.6f}')

print('=' * 80)


SAVING BEST HYPERPARAMETERS

🏆 Best Trial: 35
   Best mAP@0.5: 0.5771

📋 Best Hyperparameters:
   imgsz: 768
   optimizer: SGD
   lr0: 0.0001614596240725037
   momentum: 0.9098500376932075
   weight_decay: 0.00042786651935909105
   warmup_epochs: 1
   warmup_momentum: 0.5149307497845836
   warmup_bias_lr: 0.018938388657529233
   mosaic: 0.8205379408033115
   mixup: 0.023789168654792303

✓ Best hyperparameters saved to: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/best_hyperparameters.json
✓ Best hyperparameters saved to: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/best_hyperparameters.yaml

📋 Best Hyperparameters Summary:
  Optimizer: SGD
  Learning Rate: 0.000161
  Momentum: 0.9099
  Weight Decay: 0.000428


In [26]:
# ============================================================================
# INSTALL AND VERIFY KALEIDO (Required for PNG export of Plotly figures)
# ============================================================================
# Run this cell to ensure kaleido is installed correctly

import subprocess
import sys

# Force reinstall kaleido with a specific compatible version
print('📦 Installing kaleido (this may take a moment)...')
result = subprocess.run(
    [sys.executable, '-m', 'pip', 'install', '--upgrade', '--force-reinstall', 'kaleido==0.2.1'],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print('✓ Kaleido 0.2.1 installed successfully')
else:
    print(f'⚠️  Installation warning: {result.stderr}')

# Try importing kaleido
try:
    import kaleido
    print('✓ Kaleido module imported')
except ImportError as e:
    print(f'❌ Failed to import kaleido: {e}')
    print('   Please restart the runtime: Runtime > Restart Runtime')

# Verify kaleido works with plotly
print('\n🧪 Testing kaleido with Plotly...')
try:
    import plotly.graph_objects as go
    import plotly.io as pio

    # Create a simple test figure
    test_fig = go.Figure(data=[go.Scatter(x=[1, 2, 3], y=[1, 2, 3])])

    # Try to convert to PNG bytes (doesn't write to disk)
    img_bytes = test_fig.to_image(format="png", width=100, height=100, engine="kaleido")
    print(f'✓ Kaleido is working correctly! (Generated {len(img_bytes)} bytes)')
    print('✅ PNG export is ready to use')

except Exception as e:
    print(f'❌ Kaleido test failed: {type(e).__name__}')
    print(f'   Error: {e}')
    print('\n⚠️  ACTION REQUIRED:')
    print('   1. Go to: Runtime > Restart Runtime')
    print('   2. After restart, run all cells again')
    print('   3. Kaleido should work after the runtime restart')


📦 Installing kaleido (this may take a moment)...
✓ Kaleido 0.2.1 installed successfully
✓ Kaleido module imported

🧪 Testing kaleido with Plotly...
❌ Kaleido test failed: ValueError
   Error: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


⚠️  ACTION REQUIRED:
   1. Go to: Runtime > Restart Runtime
   2. After restart, run all cells again
   3. Kaleido should work after the runtime restart


In [27]:
# ============================================================================
# WORKAROUND: Convert Plotly figures to PNG without kaleido
# ============================================================================
# This cell provides an alternative to save PNG images when kaleido doesn't work

def save_plotly_as_png_alternative(fig, output_path, width=1200, height=800):
    """
    Alternative method to save Plotly figure as PNG without kaleido.
    Uses matplotlib as a fallback by converting through static image.
    """
    try:
        # Method 1: Try orca (older engine, might be available)
        try:
            fig.write_image(str(output_path), width=width, height=height, scale=2, engine="orca")
            return True, "orca"
        except:
            pass

        # Method 2: Save as SVG then convert (requires cairosvg)
        try:
            import cairosvg
            svg_path = str(output_path).replace('.png', '_temp.svg')
            fig.write_image(svg_path, width=width, height=height, format='svg')
            cairosvg.svg2png(url=svg_path, write_to=str(output_path), output_width=width, output_height=height)
            os.remove(svg_path)
            return True, "svg+cairosvg"
        except:
            pass

        # Method 3: Use selenium/chrome (Colab has chrome)
        try:
            import plotly.io as pio
            pio.kaleido.scope.chromium_args = tuple([arg for arg in pio.kaleido.scope.chromium_args if arg != "--disable-dev-shm-usage"])
            fig.write_image(str(output_path), width=width, height=height, scale=2)
            return True, "kaleido-fixed"
        except:
            pass

        # Method 4: Just save high-quality HTML (can be converted later)
        html_path = str(output_path).replace('.png', '_hq.html')
        fig.write_html(
            html_path,
            config={'toImageButtonOptions': {'format': 'png', 'width': width, 'height': height, 'scale': 2}}
        )
        print(f'   ℹ️  Saved high-quality HTML instead: {html_path}')
        print(f'      You can open it and use the camera icon to download PNG')
        return False, "html-fallback"

    except Exception as e:
        print(f'   ❌ All conversion methods failed: {e}')
        return False, "failed"

print('✓ PNG conversion workaround functions loaded')
print('  Use save_plotly_as_png_alternative(fig, path) to save PNG files')


✓ PNG conversion workaround functions loaded
  Use save_plotly_as_png_alternative(fig, path) to save PNG files


In [28]:
# ============================================================================
# SIMPLE SOLUTION: Manually download PNGs from the interactive plots above
# ============================================================================
# Since kaleido isn't working, here's what to do:
#
# 1. Scroll up to the interactive Plotly visualizations displayed above
# 2. Hover over each plot and you'll see a camera icon in the top-right
# 3. Click the camera icon to download the PNG file
# 4. The files will be saved to your Downloads folder
# 5. Upload them to your Colab files or Drive if needed
#
# Or, use this cell to get download links for the HTML files:

print('📊 Your visualization files:')
print('=' * 80)

# Find the latest HTML files
import glob
from pathlib import Path

tune_dir = Path(TUNE_DIR)
html_files = {
    'Optimization History': sorted(tune_dir.glob('optimization_history_*.html'))[-1:],
    'Parameter Importance': sorted(tune_dir.glob('parameter_importance_*.html'))[-1:],
    'Parameter Slice': sorted(tune_dir.glob('parameter_slice_*.html'))[-1:]
}

for title, files in html_files.items():
    if files:
        file_path = files[0]
        print(f'\n{title}:')
        print(f'  📁 {file_path}')
        print(f'  💡 Open this file in Colab and click the camera icon to download PNG')

print('\n' + '=' * 80)
print('📝 To download PNG files:')
print('  1. Open each HTML file by double-clicking it in the Files panel')
print('  2. The interactive plot will open in a new tab')
print('  3. Hover over the plot and click the camera icon (📷) in the toolbar')
print('  4. The PNG will download automatically')
print('\n💡 Alternative: Run the plots again after restarting runtime')
print('   (Your study results are saved in the .pkl file, so they won\'t be lost!)')


📊 Your visualization files:

Optimization History:
  📁 /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/optimization_history_20251128_203559.html
  💡 Open this file in Colab and click the camera icon to download PNG

Parameter Importance:
  📁 /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/parameter_importance_20251128_203559.html
  💡 Open this file in Colab and click the camera icon to download PNG

Parameter Slice:
  📁 /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/parameter_slice_20251128_203559.html
  💡 Open this file in Colab and click the camera icon to download PNG

📝 To download PNG files:
  1. Open each HTML file by double-clicking it in the Files panel
  2. The interactive plot will open in a new tab
  3. Hover over the plot and click the camera icon (📷) in 

## 11. Visualize Optimization Results

In [29]:
# ============================================================================
# VISUALIZE OPTIMIZATION RESULTS: HISTORY, PARAMETER IMPORTANCE, SLICE PLOTS
# ============================================================================

print('\n' + '=' * 80)
print('GENERATING OPTIMIZATION VISUALIZATIONS')
print('=' * 80)

if len(study.trials) == 0:
    print("⚠️  No trials found in study, skipping visualization.")
else:
    timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')

    # -----------------------------
    # 1️⃣ Optimization History Plot
    # -----------------------------
    try:
        print('\n📈 Creating optimization history plot...')
        fig_history = plot_optimization_history(study)
        fig_history.update_layout(
            title=f'{MODEL_NAME} - Hyperparameter Optimization History',
            xaxis_title='Trial Number',
            yaxis_title='mAP@0.5',
            template='plotly_white',
            width=1200,
            height=600
        )
        fig_history.show()

        # Save HTML with timestamp
        optimization_history_path = TUNE_DIR / f'optimization_history_{timestamp_str}.html'
        fig_history.write_html(str(optimization_history_path))
        print(f'✓ HTML saved to: {optimization_history_path}')

    except Exception as history_error:
        print(f'❌ Failed to create optimization history plot: {history_error}')

    # -----------------------------
    # 2️⃣ Parameter Importance Plot
    # -----------------------------
    try:
        print('\n📊 Creating parameter importance plot...')
        fig_importance = plot_param_importances(study)
        fig_importance.update_layout(
            title=f'{MODEL_NAME} - Hyperparameter Importance',
            xaxis_title='Importance',
            yaxis_title='Parameter',
            template='plotly_white',
            width=1200,
            height=800
        )
        fig_importance.show()

        # Save HTML with timestamp
        param_importance_path = TUNE_DIR / f'parameter_importance_{timestamp_str}.html'
        fig_importance.write_html(str(param_importance_path))
        print(f'✓ HTML saved to: {param_importance_path}')

        # Save PNG with timestamp AND consistent name
        try:
            # Try kaleido first
            param_importance_img_ts = TUNE_DIR / f'parameter_importance_{timestamp_str}.png'
            fig_importance.write_image(str(param_importance_img_ts), width=1200, height=800, scale=2)
            print(f'✓ PNG saved to: {param_importance_img_ts}')

            # Consistent name for PDF report
            param_importance_img = TUNE_DIR / 'parameter_importance.png'
            fig_importance.write_image(str(param_importance_img), width=1200, height=800, scale=2)
            print(f'✓ PNG saved to: {param_importance_img} (for PDF report)')
        except Exception as png_error:
            print(f'⚠️  Could not save PNG: {png_error}')
            print(f'   Error type: {type(png_error).__name__}')
            import traceback
            print(f'   Details: {traceback.format_exc()}')
            param_importance_img = None

    except (RuntimeError, ValueError) as importance_error:
        print(f'⚠️  Could not generate parameter importance plot: {importance_error}')
        print('  (This can happen when trials have insufficient data variation)')
        param_importance_img = None

    # -----------------------------
    # 3️⃣ Parameter Slice Plots
    # -----------------------------
    try:
        print('\n🔍 Creating parameter slice plots...')
        fig_slice = plot_slice(study)
        fig_slice.update_layout(
            title=f'{MODEL_NAME} - Parameter Slice Plot',
            template='plotly_white',
            width=1400,
            height=1000
        )
        fig_slice.show()

        # Save HTML with timestamp
        slice_path = TUNE_DIR / f'parameter_slice_{timestamp_str}.html'
        fig_slice.write_html(str(slice_path))
        print(f'✓ HTML saved to: {slice_path}')

        # Save PNG with timestamp AND consistent name
        try:
            # Try kaleido
            slice_img_path_ts = TUNE_DIR / f'parameter_slice_{timestamp_str}.png'
            fig_slice.write_image(str(slice_img_path_ts), width=1400, height=1000, scale=2)
            print(f'✓ PNG saved to: {slice_img_path_ts}')

            # Consistent name for PDF report
            slice_img_path = TUNE_DIR / 'parameter_slice.png'
            fig_slice.write_image(str(slice_img_path), width=1400, height=1000, scale=2)
            print(f'✓ PNG saved to: {slice_img_path} (for PDF report)')
        except Exception as png_error:
            print(f'⚠️  Could not save PNG: {png_error}')
            print(f'   Error type: {type(png_error).__name__}')
            import traceback
            print(f'   Details: {traceback.format_exc()}')

    except Exception as slice_error:
        print(f'⚠️  Could not generate parameter slice plot: {slice_error}')


GENERATING OPTIMIZATION VISUALIZATIONS

📈 Creating optimization history plot...


✓ HTML saved to: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/optimization_history_20251128_204426.html

📊 Creating parameter importance plot...


✓ HTML saved to: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/parameter_importance_20251128_204426.html
⚠️  Could not save PNG: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

   Error type: ValueError
   Details: Traceback (most recent call last):
  File "/tmp/ipython-input-706232807.py", line 63, in <cell line: 0>
    fig_importance.write_image(str(param_importance_img_ts), width=1200, height=800, scale=2)
  File "/usr/local/lib/python3.12/dist-packages/plotly/basedatatypes.py", line 3835, in write_image
    return pio.write_image(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/plotly/io/_kaleido.py", line 266, in write_image
    img_data = to_image(
               ^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/plotly/io/_kaleido.py", line 

✓ HTML saved to: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340/parameter_slice_20251128_204426.html
⚠️  Could not save PNG: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

   Error type: ValueError
   Details: Traceback (most recent call last):
  File "/tmp/ipython-input-706232807.py", line 105, in <cell line: 0>
    fig_slice.write_image(str(slice_img_path_ts), width=1400, height=1000, scale=2)
  File "/usr/local/lib/python3.12/dist-packages/plotly/basedatatypes.py", line 3835, in write_image
    return pio.write_image(self, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/plotly/io/_kaleido.py", line 266, in write_image
    img_data = to_image(
               ^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/plotly/io/_kaleido.py", line 132, in to_ima

## 12. Generate Tuning PDF Report

Create a comprehensive PDF report with optimization results, visualizations, and model performance.

In [42]:
# GENERATE Tuning PDF REPORT
# ============================================================================

print('\n' + '=' * 80)
print('GENERATING COMPREHENSIVE TUNING PDF REPORT')
print('=' * 80)

# Use already extracted best parameters and trial data from previous sections
best_params = study.best_params
best_trial = study.best_trial

print(f'\n📊 Preparing comprehensive report with {len(study.trials)} trials')
print(f'   Best Trial: {best_trial.number}')
print(f'   Best mAP@0.5: {study.best_value:.4f}')

# Compile all trials data into DataFrame for PDF report
print('\n📋 Compiling trials data for report...')
trials_data_for_pdf = []

for trial in study.trials:
    # Create row with trial info and hyperparameters directly from trial.params
    row_data = {
        'trial': trial.number,
        'state': trial.state.name,
        'mAP@0.5': trial.value if trial.value is not None else 0.0,
    }

    # Add all hyperparameters directly from trial.params
    row_data.update(trial.params)

    trials_data_for_pdf.append(row_data)

# Create DataFrame and sort by mAP@0.5
df_trials = pd.DataFrame(trials_data_for_pdf)
df_trials_sorted = df_trials.sort_values('mAP@0.5', ascending=False)

print(f'✓ Compiled {len(df_trials)} trials for report')
print(f'   Available columns: {list(df_trials.columns)}')

# Create tuning PDF report
pdf_report_path = TUNE_DIR / f'{MODEL_NAME}_tuning_report.pdf'

doc = SimpleDocTemplate(str(pdf_report_path), pagesize=A4,
                       rightMargin=30, leftMargin=30,
                       topMargin=30, bottomMargin=30)

story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=18,
    textColor=rl_colors.HexColor('#2c3e50'),
    spaceAfter=30,
    alignment=TA_CENTER
)

heading_style = ParagraphStyle(
    'CustomHeading',
    parent=styles['Heading2'],
    fontSize=16,
    textColor=rl_colors.HexColor('#34495e'),
    spaceAfter=12,
    spaceBefore=20
)

small_style = ParagraphStyle(
    'SmallText',
    parent=styles['Normal'],
    fontSize=7,
    wordWrap='CJK'
)

# Title
story.append(Paragraph(f'{MODEL_NAME} Hyperparameter Tuning Report', title_style))
story.append(Paragraph(f'Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', styles['Normal']))
story.append(Spacer(1, 20))

# ===== SECTION 1: OVERVIEW =====
story.append(Paragraph('1. Optimization Overview', heading_style))

info_data = [
    ['Property', 'Value'],
    ['Model', MODEL_NAME],
    ['Dataset', YOLO_DATASET_ROOT.name],
    ['Total Trials', str(len(study.trials))],
    ['Completed Trials', str(len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]))],
    ['Failed Trials', str(len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL]))],
    ['Best Trial', str(study.best_trial.number)],
    ['Best mAP@0.5', f'{study.best_value:.4f}'],
    ['Optimization Duration', str(duration)],
]

info_table = Table(info_data, colWidths=[2.5*inch, 3.5*inch])
info_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#2c3e50')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('BACKGROUND', (0, 1), (-1, -1), rl_colors.HexColor('#ecf0f1')),
    ('TEXTCOLOR', (0, 1), (-1, -1), rl_colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTNAME', (0, 1), (0, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.grey)
]))
story.append(info_table)
story.append(Spacer(1, 20))

# ===== SECTION 2: CONFIGURATION =====
story.append(Paragraph('2. Optimization Configuration', heading_style))

opt_config_data = [
    ['Parameter', 'Value'],
    ['Total Trials', str(N_TRIALS)],
    ['Epochs per Trial', str(EPOCHS_PER_TRIAL)],
    ['Batch Size', str(BATCH_SIZE)],
    ['Startup Trials (TPE)', str(N_STARTUP_TRIALS)],
    ['Device', device],
    ['Number of Classes', str(NUM_CLASSES)],
    ['Train Images', str(dataset_stats.get('train', {}).get('images', 'N/A'))],
    ['Val Images', str(dataset_stats.get('val', {}).get('images', 'N/A'))],
]

opt_config_table = Table(opt_config_data, colWidths=[3*inch, 3*inch])
opt_config_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#95a5a6')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(opt_config_table)
story.append(Spacer(1, 20))

# ===== SECTION 3: BEST HYPERPARAMETERS =====
story.append(PageBreak())
story.append(Paragraph('3. Best Hyperparameters', heading_style))

hyperparam_data = [['Parameter', 'Value', 'Description']]
param_descriptions = {
    'optimizer': 'Optimization algorithm',
    'lr0': 'Initial learning rate',
    'lrf': 'Final learning rate factor',
    'momentum': 'SGD momentum / Adam beta1',
    'weight_decay': 'Weight decay (L2 penalty)',
    'warmup_epochs': 'Warmup epochs',
    'warmup_momentum': 'Warmup momentum',
    'box': 'Box loss gain',
    'cls': 'Classification loss gain',
    'dfl': 'Distribution focal loss gain',
    'hsv_h': 'HSV-Hue augmentation',
    'hsv_s': 'HSV-Saturation augmentation',
    'hsv_v': 'HSV-Value augmentation',
    'degrees': 'Rotation augmentation',
    'translate': 'Translation augmentation',
    'scale': 'Scale augmentation',
    'shear': 'Shear augmentation',
    'perspective': 'Perspective augmentation',
    'flipud': 'Vertical flip probability',
    'fliplr': 'Horizontal flip probability',
    'mosaic': 'Mosaic augmentation',
    'mixup': 'Mixup augmentation',
    'copy_paste': 'Copy-paste augmentation',
}

for param_key, param_value in best_params.items():
    desc = param_descriptions.get(param_key, '')
    formatted_value = f'{param_value:.6f}' if isinstance(param_value, float) else str(param_value)
    hyperparam_data.append([param_key, formatted_value, desc])

hyperparam_table = Table(hyperparam_data, colWidths=[1.8*inch, 1.5*inch, 2.7*inch])
hyperparam_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (1, -1), 'CENTER'),
    ('ALIGN', (2, 1), (2, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 10),
    ('FONTSIZE', (0, 1), (-1, -1), 8),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 5),
    ('TOPPADDING', (0, 0), (-1, -1), 5),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
]))
story.append(hyperparam_table)
story.append(Spacer(1, 20))

# ===== SECTION 4: TOP 20 TRIALS WITH HYPERPARAMETERS =====
story.append(PageBreak())
story.append(Paragraph('4. Top 20 Trials Performance', heading_style))

# Create detailed top trials table with key hyperparameters
print(f'   DataFrame columns: {list(df_trials_sorted.columns)}')
print(f'   Sample row keys: {list(df_trials_sorted.head(1).iloc[0].keys())}')

top_trials_data = [['#', 'mAP@0.5', 'ImgSz', 'Opt', 'lr0', 'mom', 'mixup', 'mosaic']]
for idx, (_, row) in enumerate(df_trials_sorted.head(20).iterrows(), 1):
    # Use pd.notna() to check if value exists and is not NaN
    img_val = str(int(row['imgsz'])) if 'imgsz' in row and pd.notna(row['imgsz']) else 'N/A'
    opt_val = row.get('optimizer', 'N/A')
    lr0_val = f"{row['lr0']:.4f}" if 'lr0' in row and pd.notna(row['lr0']) else 'N/A'
    mom_val = f"{row['momentum']:.3f}" if 'momentum' in row and pd.notna(row['momentum']) else 'N/A'
    mix_val = f"{row['mixup']:.2f}" if 'mixup' in row and pd.notna(row['mixup']) else 'N/A'
    mos_val = f"{row['mosaic']:.2f}" if 'mosaic' in row and pd.notna(row['mosaic']) else 'N/A'

    top_trials_data.append([
        str(idx),
        f"{row['mAP@0.5']:.4f}",
        img_val,
        str(opt_val)[:4] if opt_val != 'N/A' else 'N/A',
        lr0_val,
        mom_val,
        mix_val,
        mos_val,
    ])

top_trials_table = Table(top_trials_data, colWidths=[0.3*inch, 0.8*inch, 0.6*inch, 0.6*inch, 0.7*inch, 0.7*inch, 0.7*inch, 0.7*inch])
top_trials_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#27ae60')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 9),
    ('FONTSIZE', (0, 1), (-1, -1), 7),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 4),
    ('TOPPADDING', (0, 0), (-1, -1), 4),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 0.5, rl_colors.black)
]))
story.append(top_trials_table)
story.append(Spacer(1, 15))

# Detailed hyperparameters for top 5 trials
story.append(PageBreak())
story.append(Paragraph('4.1 Detailed Hyperparameters - Top 5 Trials', heading_style))

print(f'   Creating detailed params for top 5 trials...')
for rank, (_, row) in enumerate(df_trials_sorted.head(5).iterrows(), 1):
    story.append(Paragraph(f'<b>Rank {rank}: Trial {int(row["trial"])} (mAP@0.5: {row["mAP@0.5"]:.4f})</b>', styles['Normal']))

    trial_params_text = []
    # Get all parameter columns (exclude trial, state, mAP@0.5)
    param_cols = [col for col in df_trials_sorted.columns if col not in ['trial', 'state', 'mAP@0.5']]

    for param_key in sorted(param_cols):
        if param_key in row and pd.notna(row[param_key]):
            value = row[param_key]
            formatted_val = f'{value:.6f}' if isinstance(value, float) else str(value)
            trial_params_text.append(f'{param_key}={formatted_val}')

    if trial_params_text:
        params_str = ', '.join(trial_params_text)
        story.append(Paragraph(params_str, small_style))
    else:
        story.append(Paragraph('No parameter data available', small_style))
    story.append(Spacer(1, 10))

print(f'   ✓ Top 5 trials details added')

# ===== SECTION 5: OPTIMIZATION VISUALIZATIONS =====
story.append(PageBreak())
story.append(Paragraph('5. Optimization Visualizations', heading_style))

print('\n📊 Generating custom visualizations for PDF report...')

# Prepare data for completed trials only
completed_trials_df = df_trials_sorted[df_trials_sorted['state'] == 'COMPLETE'].copy()

print(f'   Completed trials: {len(completed_trials_df)}')
print(f'   Columns available: {list(completed_trials_df.columns)}')

if len(completed_trials_df) == 0:
    story.append(Paragraph('No completed trials available for visualization.', styles['Normal']))
    print('   ⚠️ No completed trials found!')
else:
    # 5.1 mAP@0.5 Progress Over Trials
    story.append(Paragraph('5.1 mAP@0.5 Progress Over Trials', styles['Heading3']))

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(completed_trials_df['trial'], completed_trials_df['mAP@0.5'],
            marker='o', linestyle='-', linewidth=2, markersize=6, color='#3498db', alpha=0.7)
    ax.axhline(y=study.best_value, color='#e74c3c', linestyle='--', linewidth=2,
               label=f'Best: {study.best_value:.4f}')
    ax.set_xlabel('Trial Number', fontsize=12, fontweight='bold')
    ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
    ax.set_title(f'{MODEL_NAME} - mAP@0.5 Progress', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    plt.tight_layout()

    map_progress_img = TUNE_DIR / 'report_map_progress.png'
    plt.savefig(map_progress_img, dpi=150, bbox_inches='tight')
    plt.close()

    story.append(Image(str(map_progress_img), width=6.5*inch, height=3.25*inch))
    story.append(Spacer(1, 15))
    print(f'✓ mAP progress chart saved: {map_progress_img}')

    # 5.2 Learning Rate vs mAP@0.5
    story.append(PageBreak())
    story.append(Paragraph('5.2 Learning Rate Impact on Performance', styles['Heading3']))

    if 'lr0' in completed_trials_df.columns and completed_trials_df['lr0'].notna().any():
        print(f'   Creating learning rate impact chart...')
        fig, ax = plt.subplots(figsize=(10, 5))
        scatter = ax.scatter(completed_trials_df['lr0'], completed_trials_df['mAP@0.5'],
                           c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                           s=100, alpha=0.6, edgecolors='black', linewidth=0.5)
        ax.set_xlabel('Learning Rate (lr0)', fontsize=12, fontweight='bold')
        ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
        ax.set_title(f'{MODEL_NAME} - Learning Rate vs Performance', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('mAP@0.5', fontsize=10)
        plt.tight_layout()

        lr_impact_img = TUNE_DIR / 'report_lr_impact.png'
        plt.savefig(lr_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(lr_impact_img), width=6.5*inch, height=3.25*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Learning rate impact chart saved: {lr_impact_img}')
    else:
        story.append(Paragraph('Learning rate data not available for visualization.', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ lr0 column not found or empty')

    # 5.3 Optimizer Comparison
    story.append(PageBreak())
    story.append(Paragraph('5.3 Optimizer Performance Comparison', styles['Heading3']))

    if 'optimizer' in completed_trials_df.columns and completed_trials_df['optimizer'].notna().any():
        print(f'   Creating optimizer comparison chart...')
        fig, ax = plt.subplots(figsize=(12, 6))

        # Calculate comprehensive statistics
        optimizer_stats = completed_trials_df.groupby('optimizer')['mAP@0.5'].agg(['mean', 'max', 'min', 'std', 'count'])
        optimizer_stats = optimizer_stats.sort_values('mean', ascending=False)

        x_pos = range(len(optimizer_stats))

        # Create bars with gradient effect
        bars = ax.bar(x_pos, optimizer_stats['mean'], alpha=0.8,
                     color=['#2ecc71', '#3498db', '#9b59b6', '#e67e22'][:len(optimizer_stats)],
                     edgecolor='black', linewidth=1.5, width=0.6)

        # Add max and min markers
        ax.scatter(x_pos, optimizer_stats['max'], color='#27ae60', s=150,
                  label='Max mAP@0.5', zorder=5, edgecolors='black', linewidth=1.5, marker='^')
        ax.scatter(x_pos, optimizer_stats['min'], color='#e74c3c', s=150,
                  label='Min mAP@0.5', zorder=5, edgecolors='black', linewidth=1.5, marker='v')

        # Add error bars for standard deviation
        ax.errorbar(x_pos, optimizer_stats['mean'], yerr=optimizer_stats['std'],
                   fmt='none', ecolor='gray', alpha=0.5, capsize=5, capthick=2, linewidth=2)

        ax.set_xlabel('Optimizer Type', fontsize=13, fontweight='bold')
        ax.set_ylabel('mAP@0.5', fontsize=13, fontweight='bold')
        ax.set_title(f'{MODEL_NAME} - Optimizer Performance Comparison (Mean ± Std Dev)',
                    fontsize=14, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels([])
        ax.legend(fontsize=11, loc='lower left', framealpha=0.9, ncol=2)
        ax.grid(True, alpha=0.3, axis='y', linestyle='--')

        # Add optimizer names inside bars
        for i, (opt, row) in enumerate(optimizer_stats.iterrows()):
            # Optimizer name inside bar (centered vertically)
            ax.text(i, row['mean'] / 2, opt.upper(),
                   ha='center', va='center', fontsize=12, fontweight='bold',
                   color='white', rotation=0)

            # Mean value below optimizer name in bar
            ax.text(i, row['mean'] / 2 - 0.02, f"{row['mean']:.4f}",
                   ha='center', va='top', fontsize=9, fontweight='bold',
                   color='white', alpha=0.9)

            # Trial count above max point
            ax.text(i, row['max'] - 0.05, f"n={int(row['count'])}",
                   ha='center', va='bottom', fontsize=10, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7, edgecolor='black'))

        plt.tight_layout()

        optimizer_comp_img = TUNE_DIR / 'report_optimizer_comparison.png'
        plt.savefig(optimizer_comp_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(optimizer_comp_img), width=6.5*inch, height=3.25*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Optimizer comparison chart saved: {optimizer_comp_img}')

        # Add detailed statistics table for optimizers
        optimizer_table_data = [['Optimizer', 'Mean', 'Max', 'Min', 'Std Dev', 'Trials']]
        for opt, row in optimizer_stats.iterrows():
            optimizer_table_data.append([
                opt.upper(),
                f"{row['mean']:.4f}",
                f"{row['max']:.4f}",
                f"{row['min']:.4f}",
                f"{row['std']:.4f}",
                str(int(row['count']))
            ])

        opt_table = Table(optimizer_table_data, colWidths=[1.2*inch, 1.0*inch, 1.0*inch, 1.0*inch, 1.0*inch, 0.8*inch])
        opt_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
            ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
            ('TOPPADDING', (0, 0), (-1, -1), 6),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
            ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
        ]))
        story.append(opt_table)
        story.append(Spacer(1, 15))

        # Add interpretation text
        best_optimizer = optimizer_stats.index[0]
        best_mean = optimizer_stats.iloc[0]['mean']
        interpretation = f"<b>Analysis:</b> {best_optimizer.upper()} achieved the highest mean performance ({best_mean:.4f}) across {int(optimizer_stats.iloc[0]['count'])} trials. The error bars show the standard deviation, indicating performance consistency."
        story.append(Paragraph(interpretation, styles['Normal']))
        story.append(Spacer(1, 15))
    else:
        story.append(Paragraph('Optimizer data not available for visualization.', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ optimizer column not found or empty')

    # 5.4 Augmentation Parameters vs Performance
    story.append(PageBreak())
    story.append(Paragraph('5.4 Augmentation Parameters Impact', styles['Heading3']))

    # Create 2x2 subplot for key augmentation parameters
    aug_params = ['mixup', 'mosaic', 'degrees', 'scale']
    available_aug_params = [p for p in aug_params if p in completed_trials_df.columns and completed_trials_df[p].notna().any()]

    print(f'   Available augmentation params: {available_aug_params}')

    if len(available_aug_params) >= 2:
        n_plots = min(len(available_aug_params), 4)
        fig, axes = plt.subplots(2, 2, figsize=(10, 8))
        axes = axes.flatten()

        for idx, param in enumerate(available_aug_params[:4]):
            ax = axes[idx]
            scatter = ax.scatter(completed_trials_df[param], completed_trials_df['mAP@0.5'],
                               c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                               s=60, alpha=0.6, edgecolors='black', linewidth=0.5)
            ax.set_xlabel(param, fontsize=10, fontweight='bold')
            ax.set_ylabel('mAP@0.5', fontsize=10, fontweight='bold')
            ax.set_title(f'{param.capitalize()} Impact', fontsize=11, fontweight='bold')
            ax.grid(True, alpha=0.3)

        # Hide unused subplots
        for idx in range(len(available_aug_params), 4):
            axes[idx].axis('off')

        plt.tight_layout()

        aug_impact_img = TUNE_DIR / 'report_augmentation_impact.png'
        plt.savefig(aug_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(aug_impact_img), width=6.5*inch, height=5.2*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Augmentation impact chart saved: {aug_impact_img}')
    else:
        story.append(Paragraph(f'Insufficient augmentation parameter data for visualization. Found: {available_aug_params}', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ Not enough augmentation params available')

    # 5.5 Weight Decay and Momentum vs Performance
    story.append(PageBreak())
    story.append(Paragraph('5.5 Regularization Parameters Impact', styles['Heading3']))

    has_weight_decay = 'weight_decay' in completed_trials_df.columns and completed_trials_df['weight_decay'].notna().any()
    has_momentum = 'momentum' in completed_trials_df.columns and completed_trials_df['momentum'].notna().any()

    if has_weight_decay and has_momentum:
        print(f'   Creating regularization impact chart...')
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

        # Weight Decay
        scatter1 = ax1.scatter(completed_trials_df['weight_decay'], completed_trials_df['mAP@0.5'],
                              c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                              s=80, alpha=0.6, edgecolors='black', linewidth=0.5)
        ax1.set_xlabel('Weight Decay', fontsize=11, fontweight='bold')
        ax1.set_ylabel('mAP@0.5', fontsize=11, fontweight='bold')
        ax1.set_title('Weight Decay Impact', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)

        # Momentum
        scatter2 = ax2.scatter(completed_trials_df['momentum'], completed_trials_df['mAP@0.5'],
                              c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                              s=80, alpha=0.6, edgecolors='black', linewidth=0.5)
        ax2.set_xlabel('Momentum', fontsize=11, fontweight='bold')
        ax2.set_ylabel('mAP@0.5', fontsize=11, fontweight='bold')
        ax2.set_title('Momentum Impact', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()

        reg_impact_img = TUNE_DIR / 'report_regularization_impact.png'
        plt.savefig(reg_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(reg_impact_img), width=6.5*inch, height=2.6*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Regularization impact chart saved: {reg_impact_img}')
    else:
        story.append(Paragraph(f'Regularization parameter data not available. weight_decay: {has_weight_decay}, momentum: {has_momentum}', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ weight_decay or momentum columns not found or empty')

    # 5.6 Image Size Impact on Performance
    story.append(PageBreak())
    story.append(Paragraph('5.6 Image Size Impact on Performance', styles['Heading3']))

    if 'imgsz' in completed_trials_df.columns and completed_trials_df['imgsz'].notna().any():
        print(f'   Creating image size impact chart...')
        fig, ax = plt.subplots(figsize=(10, 5))

        # Group by image size and calculate statistics
        imgsz_stats = completed_trials_df.groupby('imgsz')['mAP@0.5'].agg(['mean', 'max', 'min', 'count'])
        imgsz_stats = imgsz_stats.sort_index()

        x_pos = range(len(imgsz_stats))
        bars = ax.bar(x_pos, imgsz_stats['mean'], alpha=0.7, color='#9b59b6',
               label='Mean mAP@0.5', edgecolor='black', linewidth=1.5, width=0.6)
        ax.scatter(x_pos, imgsz_stats['max'], color='#27ae60', s=120,
                  label='Max mAP@0.5', zorder=5, edgecolors='black', linewidth=1, marker='^')
        ax.scatter(x_pos, imgsz_stats['min'], color='#e74c3c', s=120,
                  label='Min mAP@0.5', zorder=5, edgecolors='black', linewidth=1, marker='v')

        ax.set_xlabel('Image Size (pixels)', fontsize=12, fontweight='bold')
        ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
        ax.set_title(f'{MODEL_NAME} - Image Size Impact on Performance', fontsize=14, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels([int(idx) for idx in imgsz_stats.index], fontsize=11, fontweight='bold')
        ax.legend(fontsize=10, loc='best')
        ax.grid(True, alpha=0.3, axis='y')

        # Add count and mean value annotations
        for i, (imgsz, row) in enumerate(imgsz_stats.iterrows()):
            # Mean value inside bar
            ax.text(i, row['mean'] / 2, f"{row['mean']:.4f}",
                   ha='center', va='center', fontsize=10, fontweight='bold', color='white')
            # Count above bar
            ax.text(i, row['max'] + 0.003, f"n={int(row['count'])}",
                   ha='center', va='bottom', fontsize=9)

        plt.tight_layout()

        imgsz_impact_img = TUNE_DIR / 'report_imgsz_impact.png'
        plt.savefig(imgsz_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(imgsz_impact_img), width=6.5*inch, height=3.25*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Image size impact chart saved: {imgsz_impact_img}')

        # Add statistics table for image sizes
        imgsz_table_data = [['Image Size', 'Mean mAP@0.5', 'Max mAP@0.5', 'Min mAP@0.5', 'Trials']]
        for imgsz, row in imgsz_stats.iterrows():
            imgsz_table_data.append([
                str(int(imgsz)),
                f"{row['mean']:.4f}",
                f"{row['max']:.4f}",
                f"{row['min']:.4f}",
                str(int(row['count']))
            ])

        imgsz_table = Table(imgsz_table_data, colWidths=[1.2*inch, 1.2*inch, 1.2*inch, 1.2*inch, 0.8*inch])
        imgsz_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#9b59b6')),
            ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
            ('TOPPADDING', (0, 0), (-1, -1), 6),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
            ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
        ]))
        story.append(imgsz_table)
        story.append(Spacer(1, 15))
    else:
        story.append(Paragraph('Image size data not available for visualization.', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ imgsz column not found or empty')

    print('✓ All custom visualizations generated for PDF report')

# ===== SECTION 6: ALL TRIALS SUMMARY =====
story.append(PageBreak())
story.append(Paragraph('6. All Trials Summary', heading_style))

# Statistics
completed_df = df_trials_sorted[df_trials_sorted['state'] == 'COMPLETE']
if len(completed_df) > 0:
    stats_data = [
        ['Metric', 'Value'],
        ['Completed Trials', str(len(completed_df))],
        ['Best mAP@0.5', f"{completed_df['mAP@0.5'].max():.4f}"],
        ['Worst mAP@0.5', f"{completed_df['mAP@0.5'].min():.4f}"],
        ['Mean mAP@0.5', f"{completed_df['mAP@0.5'].mean():.4f}"],
        ['Std Dev mAP@0.5', f"{completed_df['mAP@0.5'].std():.4f}"],
        ['Median mAP@0.5', f"{completed_df['mAP@0.5'].median():.4f}"],
    ]

    stats_table = Table(stats_data, colWidths=[2.5*inch, 3.5*inch])
    stats_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#e74c3c')),
        ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
        ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
    ]))
    story.append(stats_table)

# Build PDF
try:
    doc.build(story)
    print(f'\n✓ Comprehensive PDF report generated: {pdf_report_path}')
    print(f'  Size: {pdf_report_path.stat().st_size / (1024*1024):.1f} MB')
    print(f'  Sections: Overview, Configuration, Best Hyperparameters, Top 20 Trials,')
    print(f'            Optimization Visualizations (6 charts), All Trials Summary')
except Exception as pdf_error:
    print(f'\n⚠️  Error generating PDF: {pdf_error}')
    import traceback
    traceback.print_exc()

print('=' * 80)


GENERATING COMPREHENSIVE TUNING PDF REPORT

📊 Preparing comprehensive report with 42 trials
   Best Trial: 35
   Best mAP@0.5: 0.5771

📋 Compiling trials data for report...
✓ Compiled 42 trials for report
   Available columns: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'warmup_epochs', 'warmup_momentum', 'warmup_bias_lr', 'mosaic', 'mixup']
   DataFrame columns: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'warmup_epochs', 'warmup_momentum', 'warmup_bias_lr', 'mosaic', 'mixup']
   Sample row keys: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'warmup_epochs', 'warmup_momentum', 'warmup_bias_lr', 'mosaic', 'mixup']
   Creating detailed params for top 5 trials...
   ✓ Top 5 trials details added

📊 Generating custom visualizations for PDF report...
   Completed trials: 41
   Columns available: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum',

In [44]:
# GENERATE Tuning PDF REPORT
# ============================================================================

print('\n' + '=' * 80)
print('GENERATING COMPREHENSIVE TUNING PDF REPORT')
print('=' * 80)

# Use already extracted best parameters and trial data from previous sections
best_params = study.best_params
best_trial = study.best_trial

print(f'\n📊 Preparing comprehensive report with {len(study.trials)} trials')
print(f'   Best Trial: {best_trial.number}')
print(f'   Best mAP@0.5: {study.best_value:.4f}')

# Compile all trials data into DataFrame for PDF report
print('\n📋 Compiling trials data for report...')
trials_data_for_pdf = []

for trial in study.trials:
    # Create row with trial info and hyperparameters directly from trial.params
    row_data = {
        'trial': trial.number,
        'state': trial.state.name,
        'mAP@0.5': trial.value if trial.value is not None else 0.0,
    }

    # Add all hyperparameters directly from trial.params
    row_data.update(trial.params)

    trials_data_for_pdf.append(row_data)

# Create DataFrame and sort by mAP@0.5
df_trials = pd.DataFrame(trials_data_for_pdf)
df_trials_sorted = df_trials.sort_values('mAP@0.5', ascending=False)

print(f'✓ Compiled {len(df_trials)} trials for report')
print(f'   Available columns: {list(df_trials.columns)}')

# Create tuning PDF report
pdf_report_path = TUNE_DIR / f'{MODEL_NAME}_tuning_report.pdf'

doc = SimpleDocTemplate(str(pdf_report_path), pagesize=A4,
                       rightMargin=30, leftMargin=30,
                       topMargin=30, bottomMargin=30)

story = []
styles = getSampleStyleSheet()

# Custom styles
title_style = ParagraphStyle(
    'CustomTitle',
    parent=styles['Heading1'],
    fontSize=18,
    textColor=rl_colors.HexColor('#2c3e50'),
    spaceAfter=30,
    alignment=TA_CENTER
)

heading_style = ParagraphStyle(
    'CustomHeading',
    parent=styles['Heading2'],
    fontSize=16,
    textColor=rl_colors.HexColor('#34495e'),
    spaceAfter=12,
    spaceBefore=20
)

small_style = ParagraphStyle(
    'SmallText',
    parent=styles['Normal'],
    fontSize=7,
    wordWrap='CJK'
)

# Title
story.append(Paragraph(f'{MODEL_NAME} Hyperparameter Tuning Report', title_style))
story.append(Paragraph(f'Generated: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', styles['Normal']))
story.append(Spacer(1, 20))

# ===== SECTION 1: OVERVIEW =====
story.append(Paragraph('1. Optimization Overview', heading_style))

info_data = [
    ['Property', 'Value'],
    ['Model', MODEL_NAME],
    ['Dataset', YOLO_DATASET_ROOT.name],
    ['Total Trials', str(len(study.trials))],
    ['Completed Trials', str(len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]))],
    ['Failed Trials', str(len([t for t in study.trials if t.state == optuna.trial.TrialState.FAIL]))],
    ['Best Trial', str(study.best_trial.number)],
    ['Best mAP@0.5', f'{study.best_value:.4f}'],
    ['Optimization Duration', str(duration)],
]

info_table = Table(info_data, colWidths=[2.5*inch, 3.5*inch])
info_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#2c3e50')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('BACKGROUND', (0, 1), (-1, -1), rl_colors.HexColor('#ecf0f1')),
    ('TEXTCOLOR', (0, 1), (-1, -1), rl_colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTNAME', (0, 1), (0, -1), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, -1), 10),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.grey)
]))
story.append(info_table)
story.append(Spacer(1, 20))

# ===== SECTION 2: CONFIGURATION =====
story.append(Paragraph('2. Optimization Configuration', heading_style))

opt_config_data = [
    ['Parameter', 'Value'],
    ['Total Trials', str(N_TRIALS)],
    ['Epochs per Trial', str(EPOCHS_PER_TRIAL)],
    ['Batch Size', str(BATCH_SIZE)],
    ['Startup Trials (TPE)', str(N_STARTUP_TRIALS)],
    ['Device', device],
    ['Number of Classes', str(NUM_CLASSES)],
    ['Train Images', str(dataset_stats.get('train', {}).get('images', 'N/A'))],
    ['Val Images', str(dataset_stats.get('val', {}).get('images', 'N/A'))],
]

opt_config_table = Table(opt_config_data, colWidths=[3*inch, 3*inch])
opt_config_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#95a5a6')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(opt_config_table)
story.append(Spacer(1, 20))

# ===== SECTION 2.5: EXECUTIVE SUMMARY & KEY FINDINGS =====
story.append(PageBreak())
story.append(Paragraph('2.5 Executive Summary & Key Findings', heading_style))

# Calculate key statistics
completed_df_summary = df_trials_sorted[df_trials_sorted['state'] == 'COMPLETE']
best_map = completed_df_summary['mAP@0.5'].max()
worst_map = completed_df_summary['mAP@0.5'].min()
mean_map = completed_df_summary['mAP@0.5'].mean()
improvement_pct = ((best_map - worst_map) / worst_map) * 100 if worst_map > 0 else 0

# Calculate optimizer statistics
if 'optimizer' in completed_df_summary.columns:
    opt_stats = completed_df_summary.groupby('optimizer')['mAP@0.5'].agg(['mean', 'count'])
    best_opt = opt_stats['mean'].idxmax()
    best_opt_mean = opt_stats.loc[best_opt, 'mean']
else:
    best_opt = 'N/A'
    best_opt_mean = 0

# Calculate image size impact
if 'imgsz' in completed_df_summary.columns:
    img_stats = completed_df_summary.groupby('imgsz')['mAP@0.5'].mean()
    best_imgsz = img_stats.idxmax()
    imgsz_improvement = ((img_stats.max() - img_stats.min()) / img_stats.min()) * 100 if len(img_stats) > 1 else 0
else:
    best_imgsz = 'N/A'
    imgsz_improvement = 0

# Create findings summary
findings_data = [
    ['Metric', 'Value'],
    ['🏆 Best Performance', f'Trial #{study.best_trial.number}: mAP@0.5 = {best_map:.4f}'],
    ['📊 Performance Range', f'{worst_map:.4f} to {best_map:.4f} ({improvement_pct:.1f}% improvement)'],
    ['📈 Mean Performance', f'{mean_map:.4f} across {len(completed_df_summary)} trials'],
    ['⚡ Best Optimizer', f'{best_opt} (mean: {best_opt_mean:.4f})'],
    ['🖼️ Optimal Image Size', f'{int(best_imgsz)}px ({imgsz_improvement:.2f}% better)' if best_imgsz != 'N/A' else 'N/A'],
    ['⏱️ Optimization Time', str(duration)],
    ['✅ Success Rate', f'{len(completed_df_summary)}/{len(study.trials)} trials ({len(completed_df_summary)/len(study.trials)*100:.1f}%)'],
]

findings_table = Table(findings_data, colWidths=[2.5*inch, 3.5*inch])
findings_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#27ae60')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('BACKGROUND', (0, 1), (-1, -1), rl_colors.HexColor('#ecf9f2')),
    ('ALIGN', (0, 0), (-1, 0), 'CENTER'),
    ('ALIGN', (0, 1), (0, -1), 'LEFT'),
    ('ALIGN', (1, 1), (1, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 11),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 8),
    ('TOPPADDING', (0, 0), (-1, -1), 8),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black),
    ('LINEBELOW', (0, 0), (-1, 0), 2, rl_colors.HexColor('#27ae60'))
]))
story.append(findings_table)
story.append(Spacer(1, 15))

# Add key insights text
insights_text = f"""
<b>Key Insights:</b><br/>
• The optimization process successfully explored {len(study.trials)} trials, achieving a {improvement_pct:.1f}% performance improvement from worst to best.<br/>
• <b>{best_opt}</b> optimizer demonstrated superior performance with mean mAP@0.5 of {best_opt_mean:.4f}.<br/>
• Image size of <b>{int(best_imgsz)}px</b> provided optimal accuracy-efficiency tradeoff.<br/>
• High consistency achieved: mean performance ({mean_map:.4f}) close to best ({best_map:.4f}), indicating robust hyperparameter space.
"""
story.append(Paragraph(insights_text, styles['Normal']))
story.append(Spacer(1, 20))

print(f'✓ Executive summary generated')

# ===== SECTION 3: BEST HYPERPARAMETERS =====
story.append(PageBreak())
story.append(Paragraph('3. Best Hyperparameters', heading_style))

hyperparam_data = [['Parameter', 'Value', 'Description']]
param_descriptions = {
    'optimizer': 'Optimization algorithm',
    'lr0': 'Initial learning rate',
    'lrf': 'Final learning rate factor',
    'momentum': 'SGD momentum / Adam beta1',
    'weight_decay': 'Weight decay (L2 penalty)',
    'warmup_epochs': 'Warmup epochs',
    'warmup_momentum': 'Warmup momentum',
    'box': 'Box loss gain',
    'cls': 'Classification loss gain',
    'dfl': 'Distribution focal loss gain',
    'hsv_h': 'HSV-Hue augmentation',
    'hsv_s': 'HSV-Saturation augmentation',
    'hsv_v': 'HSV-Value augmentation',
    'degrees': 'Rotation augmentation',
    'translate': 'Translation augmentation',
    'scale': 'Scale augmentation',
    'shear': 'Shear augmentation',
    'perspective': 'Perspective augmentation',
    'flipud': 'Vertical flip probability',
    'fliplr': 'Horizontal flip probability',
    'mosaic': 'Mosaic augmentation',
    'mixup': 'Mixup augmentation',
    'copy_paste': 'Copy-paste augmentation',
}

for param_key, param_value in best_params.items():
    desc = param_descriptions.get(param_key, '')
    formatted_value = f'{param_value:.6f}' if isinstance(param_value, float) else str(param_value)
    hyperparam_data.append([param_key, formatted_value, desc])

hyperparam_table = Table(hyperparam_data, colWidths=[1.8*inch, 1.5*inch, 2.7*inch])
hyperparam_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (1, -1), 'CENTER'),
    ('ALIGN', (2, 1), (2, -1), 'LEFT'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 10),
    ('FONTSIZE', (0, 1), (-1, -1), 8),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 5),
    ('TOPPADDING', (0, 0), (-1, -1), 5),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black),
    ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'),
]))
story.append(hyperparam_table)
story.append(Spacer(1, 20))

# ===== SECTION 4: TOP 20 TRIALS WITH HYPERPARAMETERS =====
story.append(PageBreak())
story.append(Paragraph('4. Top 20 Trials Performance', heading_style))

# Create detailed top trials table with key hyperparameters
print(f'   DataFrame columns: {list(df_trials_sorted.columns)}')
print(f'   Sample row keys: {list(df_trials_sorted.head(1).iloc[0].keys())}')

top_trials_data = [['#', 'mAP@0.5', 'ImgSz', 'Opt', 'lr0', 'mom', 'mixup', 'mosaic']]
for idx, (_, row) in enumerate(df_trials_sorted.head(20).iterrows(), 1):
    # Use pd.notna() to check if value exists and is not NaN
    img_val = str(int(row['imgsz'])) if 'imgsz' in row and pd.notna(row['imgsz']) else 'N/A'
    opt_val = row.get('optimizer', 'N/A')
    lr0_val = f"{row['lr0']:.4f}" if 'lr0' in row and pd.notna(row['lr0']) else 'N/A'
    mom_val = f"{row['momentum']:.3f}" if 'momentum' in row and pd.notna(row['momentum']) else 'N/A'
    mix_val = f"{row['mixup']:.2f}" if 'mixup' in row and pd.notna(row['mixup']) else 'N/A'
    mos_val = f"{row['mosaic']:.2f}" if 'mosaic' in row and pd.notna(row['mosaic']) else 'N/A'

    top_trials_data.append([
        str(idx),
        f"{row['mAP@0.5']:.4f}",
        img_val,
        str(opt_val)[:4] if opt_val != 'N/A' else 'N/A',
        lr0_val,
        mom_val,
        mix_val,
        mos_val,
    ])

top_trials_table = Table(top_trials_data, colWidths=[0.3*inch, 0.8*inch, 0.6*inch, 0.6*inch, 0.7*inch, 0.7*inch, 0.7*inch, 0.7*inch])
top_trials_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#27ae60')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 9),
    ('FONTSIZE', (0, 1), (-1, -1), 7),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 4),
    ('TOPPADDING', (0, 0), (-1, -1), 4),
    ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
    ('GRID', (0, 0), (-1, -1), 0.5, rl_colors.black)
]))
story.append(top_trials_table)
story.append(Spacer(1, 15))

# Detailed hyperparameters for top 5 trials
story.append(PageBreak())
story.append(Paragraph('4.1 Detailed Hyperparameters - Top 5 Trials', heading_style))

print(f'   Creating detailed params for top 5 trials...')
for rank, (_, row) in enumerate(df_trials_sorted.head(5).iterrows(), 1):
    story.append(Paragraph(f'<b>Rank {rank}: Trial {int(row["trial"])} (mAP@0.5: {row["mAP@0.5"]:.4f})</b>', styles['Normal']))

    trial_params_text = []
    # Get all parameter columns (exclude trial, state, mAP@0.5)
    param_cols = [col for col in df_trials_sorted.columns if col not in ['trial', 'state', 'mAP@0.5']]

    for param_key in sorted(param_cols):
        if param_key in row and pd.notna(row[param_key]):
            value = row[param_key]
            formatted_val = f'{value:.6f}' if isinstance(value, float) else str(value)
            trial_params_text.append(f'{param_key}={formatted_val}')

    if trial_params_text:
        params_str = ', '.join(trial_params_text)
        story.append(Paragraph(params_str, small_style))
    else:
        story.append(Paragraph('No parameter data available', small_style))
    story.append(Spacer(1, 10))

print(f'   ✓ Top 5 trials details added')

# ===== SECTION 5: OPTIMIZATION VISUALIZATIONS =====
story.append(PageBreak())
story.append(Paragraph('5. Optimization Visualizations & Analysis', heading_style))

print('\n📊 Generating custom visualizations for PDF report...')

# Prepare data for completed trials only
completed_trials_df = df_trials_sorted[df_trials_sorted['state'] == 'COMPLETE'].copy()

print(f'   Completed trials: {len(completed_trials_df)}')
print(f'   Columns available: {list(completed_trials_df.columns)}')

if len(completed_trials_df) == 0:
    story.append(Paragraph('No completed trials available for visualization.', styles['Normal']))
    print('   ⚠️ No completed trials found!')
else:
    # 5.0 Performance Distribution Box Plot
    story.append(Paragraph('5.0 Performance Distribution Analysis', styles['Heading3']))

    print(f'   Creating performance distribution box plot...')
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Box plot for overall distribution
    bp = ax1.boxplot([completed_trials_df['mAP@0.5']], vert=True, patch_artist=True,
                     labels=['All Trials'], widths=0.5)
    bp['boxes'][0].set_facecolor('#3498db')
    bp['boxes'][0].set_alpha(0.7)
    bp['medians'][0].set_color('#e74c3c')
    bp['medians'][0].set_linewidth(2)

    # Add statistics annotations
    q1 = completed_trials_df['mAP@0.5'].quantile(0.25)
    median = completed_trials_df['mAP@0.5'].median()
    q3 = completed_trials_df['mAP@0.5'].quantile(0.75)

    ax1.text(1.3, q1, f'Q1: {q1:.4f}', fontsize=9, va='center')
    ax1.text(1.3, median, f'Median: {median:.4f}', fontsize=9, va='center', fontweight='bold', color='#e74c3c')
    ax1.text(1.3, q3, f'Q3: {q3:.4f}', fontsize=9, va='center')
    ax1.text(1.3, completed_trials_df['mAP@0.5'].min(), f'Min: {completed_trials_df["mAP@0.5"].min():.4f}',
            fontsize=8, va='center', color='gray')
    ax1.text(1.3, completed_trials_df['mAP@0.5'].max(), f'Max: {completed_trials_df["mAP@0.5"].max():.4f}',
            fontsize=8, va='center', color='gray')

    ax1.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
    ax1.set_title('Overall Performance Distribution', fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='y')

    # Histogram with KDE
    ax2.hist(completed_trials_df['mAP@0.5'], bins=15, alpha=0.7, color='#3498db',
            edgecolor='black', linewidth=1)
    ax2.axvline(median, color='#e74c3c', linestyle='--', linewidth=2, label=f'Median: {median:.4f}')
    ax2.axvline(study.best_value, color='#27ae60', linestyle='--', linewidth=2, label=f'Best: {study.best_value:.4f}')
    ax2.set_xlabel('mAP@0.5', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax2.set_title('Performance Histogram', fontsize=13, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3, axis='y')

    plt.tight_layout()

    perf_dist_img = TUNE_DIR / 'report_performance_distribution.png'
    plt.savefig(perf_dist_img, dpi=150, bbox_inches='tight')
    plt.close()

    story.append(Image(str(perf_dist_img), width=6.5*inch, height=2.7*inch))
    story.append(Spacer(1, 15))
    print(f'✓ Performance distribution chart saved: {perf_dist_img}')

    # Add distribution statistics table
    dist_stats_data = [
        ['Statistic', 'Value'],
        ['Mean', f'{completed_trials_df["mAP@0.5"].mean():.4f}'],
        ['Median', f'{median:.4f}'],
        ['Std Dev', f'{completed_trials_df["mAP@0.5"].std():.4f}'],
        ['IQR (Q3-Q1)', f'{q3-q1:.4f}'],
        ['Range', f'{completed_trials_df["mAP@0.5"].max() - completed_trials_df["mAP@0.5"].min():.4f}'],
    ]

    dist_table = Table(dist_stats_data, colWidths=[2*inch, 2*inch])
    dist_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
        ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 10),
        ('FONTSIZE', (0, 1), (-1, -1), 9),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
        ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
    ]))
    story.append(dist_table)
    story.append(Spacer(1, 15))

    # 5.1 Parameter Correlation Heatmap
    story.append(PageBreak())
    story.append(Paragraph('5.1 Parameter Correlation Analysis', styles['Heading3']))

    print(f'   Creating parameter correlation heatmap...')

    # Select numeric columns for correlation
    numeric_cols = ['mAP@0.5']
    param_cols = ['lr0', 'momentum', 'weight_decay', 'mixup', 'mosaic']
    available_params = [col for col in param_cols if col in completed_trials_df.columns and completed_trials_df[col].notna().any()]

    if len(available_params) >= 2:
        corr_cols = numeric_cols + available_params
        corr_data = completed_trials_df[corr_cols].corr()

        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.imshow(corr_data, cmap='RdYlGn', aspect='auto', vmin=-1, vmax=1)

        # Set ticks and labels
        ax.set_xticks(range(len(corr_cols)))
        ax.set_yticks(range(len(corr_cols)))
        ax.set_xticklabels(corr_cols, rotation=45, ha='right', fontsize=10)
        ax.set_yticklabels(corr_cols, fontsize=10)

        # Add correlation values as text
        for i in range(len(corr_cols)):
            for j in range(len(corr_cols)):
                value = corr_data.iloc[i, j]
                color = 'white' if abs(value) > 0.5 else 'black'
                ax.text(j, i, f'{value:.2f}', ha='center', va='center',
                       color=color, fontsize=9, fontweight='bold')

        # Add colorbar
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_label('Correlation Coefficient', fontsize=11, fontweight='bold')

        ax.set_title(f'{MODEL_NAME} - Parameter Correlation with Performance',
                    fontsize=13, fontweight='bold', pad=20)
        plt.tight_layout()

        corr_img = TUNE_DIR / 'report_correlation_heatmap.png'
        plt.savefig(corr_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(corr_img), width=6*inch, height=4.8*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Correlation heatmap saved: {corr_img}')

        # Add interpretation
        map_corr = corr_data['mAP@0.5'].drop('mAP@0.5')
        strongest_pos = map_corr.idxmax() if map_corr.max() > 0 else None
        strongest_neg = map_corr.idxmin() if map_corr.min() < 0 else None

        corr_text = f"<b>Correlation Insights:</b><br/>"
        if strongest_pos:
            corr_text += f"• Strongest positive correlation: <b>{strongest_pos}</b> ({map_corr[strongest_pos]:.3f}) - Higher values tend to improve performance.<br/>"
        if strongest_neg:
            corr_text += f"• Strongest negative correlation: <b>{strongest_neg}</b> ({map_corr[strongest_neg]:.3f}) - Higher values tend to decrease performance.<br/>"
        corr_text += f"• Green cells indicate positive correlation, red cells indicate negative correlation."

        story.append(Paragraph(corr_text, styles['Normal']))
        story.append(Spacer(1, 15))

    # 5.2 Optimization Timeline & Convergence
    story.append(PageBreak())
    story.append(Paragraph('5.2 Optimization Timeline & Convergence', styles['Heading3']))

    print(f'   Creating optimization timeline chart...')
    fig, ax = plt.subplots(figsize=(12, 5))

    # Sort by trial number for timeline
    timeline_df = completed_trials_df.sort_values('trial')

    # Plot actual performance
    ax.plot(timeline_df['trial'], timeline_df['mAP@0.5'],
            marker='o', linestyle='-', linewidth=1.5, markersize=5,
            color='#95a5a6', alpha=0.5, label='Trial Performance')

    # Calculate and plot moving average (window=5)
    window = min(5, len(timeline_df))
    if window > 1:
        moving_avg = timeline_df['mAP@0.5'].rolling(window=window, min_periods=1).mean()
        ax.plot(timeline_df['trial'], moving_avg,
               linewidth=3, color='#3498db', label=f'{window}-Trial Moving Average')

    # Calculate and plot cumulative best
    cumulative_best = timeline_df['mAP@0.5'].cummax()
    ax.plot(timeline_df['trial'], cumulative_best,
           linewidth=2.5, color='#27ae60', linestyle='--',
           label='Cumulative Best', marker='*', markersize=8, markevery=cumulative_best.diff().fillna(1) != 0)

    # Mark best trial
    best_trial_idx = timeline_df[timeline_df['mAP@0.5'] == study.best_value].iloc[0]
    ax.scatter([best_trial_idx['trial']], [study.best_value],
              s=300, color='#e74c3c', marker='*', zorder=5,
              edgecolors='black', linewidth=2, label=f'Best Trial #{int(best_trial_idx["trial"])}')
    ax.annotate(f'Best: {study.best_value:.4f}',
               xy=(best_trial_idx['trial'], study.best_value),
               xytext=(10, 10), textcoords='offset points',
               fontsize=10, fontweight='bold',
               bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
               arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', lw=2))

    ax.set_xlabel('Trial Number', fontsize=12, fontweight='bold')
    ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
    ax.set_title(f'{MODEL_NAME} - Optimization Progress & Convergence', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10, loc='lower right')
    ax.grid(True, alpha=0.3)

    plt.tight_layout()

    timeline_img = TUNE_DIR / 'report_optimization_timeline.png'
    plt.savefig(timeline_img, dpi=150, bbox_inches='tight')
    plt.close()

    story.append(Image(str(timeline_img), width=6.5*inch, height=2.7*inch))
    story.append(Spacer(1, 15))
    print(f'✓ Optimization timeline chart saved: {timeline_img}')

    # Add convergence analysis
    best_found_at = int(best_trial_idx['trial'])
    total_trials = len(timeline_df)
    convergence_pct = (best_found_at / total_trials) * 100

    convergence_text = f"""<b>Convergence Analysis:</b><br/>
• Best solution found at trial <b>#{best_found_at}</b> ({convergence_pct:.1f}% through optimization).<br/>
• Moving average shows {'rapid early convergence' if convergence_pct < 40 else 'gradual improvement' if convergence_pct < 70 else 'late discovery'} pattern.<br/>
• Cumulative best curve indicates {'efficient' if convergence_pct < 50 else 'moderate'} exploration of hyperparameter space.
"""
    story.append(Paragraph(convergence_text, styles['Normal']))
    story.append(Spacer(1, 15))

    # 5.3 mAP@0.5 Progress Over Trials (original chart)
    story.append(PageBreak())
    story.append(Paragraph('5.3 mAP@0.5 Progress Over Trials', styles['Heading3']))

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(completed_trials_df['trial'], completed_trials_df['mAP@0.5'],
            marker='o', linestyle='-', linewidth=2, markersize=6, color='#3498db', alpha=0.7)
    ax.axhline(y=study.best_value, color='#e74c3c', linestyle='--', linewidth=2,
               label=f'Best: {study.best_value:.4f}')
    ax.set_xlabel('Trial Number', fontsize=12, fontweight='bold')
    ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
    ax.set_title(f'{MODEL_NAME} - mAP@0.5 Progress', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=10)
    plt.tight_layout()

    map_progress_img = TUNE_DIR / 'report_map_progress.png'
    plt.savefig(map_progress_img, dpi=150, bbox_inches='tight')
    plt.close()

    story.append(Image(str(map_progress_img), width=6.5*inch, height=3.25*inch))
    story.append(Spacer(1, 15))
    print(f'✓ mAP progress chart saved: {map_progress_img}')

    # 5.4 Learning Rate vs mAP@0.5
    story.append(PageBreak())
    story.append(Paragraph('5.4 Learning Rate Impact on Performance', styles['Heading3']))

    if 'lr0' in completed_trials_df.columns and completed_trials_df['lr0'].notna().any():
        print(f'   Creating learning rate impact chart...')
        fig, ax = plt.subplots(figsize=(10, 5))
        scatter = ax.scatter(completed_trials_df['lr0'], completed_trials_df['mAP@0.5'],
                           c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                           s=100, alpha=0.6, edgecolors='black', linewidth=0.5)
        ax.set_xlabel('Learning Rate (lr0)', fontsize=12, fontweight='bold')
        ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
        ax.set_title(f'{MODEL_NAME} - Learning Rate vs Performance', fontsize=14, fontweight='bold')
        ax.grid(True, alpha=0.3)
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('mAP@0.5', fontsize=10)
        plt.tight_layout()

        lr_impact_img = TUNE_DIR / 'report_lr_impact.png'
        plt.savefig(lr_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(lr_impact_img), width=6.5*inch, height=3.25*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Learning rate impact chart saved: {lr_impact_img}')
    else:
        story.append(Paragraph('Learning rate data not available for visualization.', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ lr0 column not found or empty')

    # 5.5 Optimizer Comparison
    story.append(PageBreak())
    story.append(Paragraph('5.5 Optimizer Performance Comparison', styles['Heading3']))

    if 'optimizer' in completed_trials_df.columns and completed_trials_df['optimizer'].notna().any():
        print(f'   Creating optimizer comparison chart...')
        fig, ax = plt.subplots(figsize=(12, 6))

        # Calculate comprehensive statistics
        optimizer_stats = completed_trials_df.groupby('optimizer')['mAP@0.5'].agg(['mean', 'max', 'min', 'std', 'count'])
        optimizer_stats = optimizer_stats.sort_values('mean', ascending=False)

        x_pos = range(len(optimizer_stats))

        # Create bars with gradient effect
        bars = ax.bar(x_pos, optimizer_stats['mean'], alpha=0.8,
                     color=['#2ecc71', '#3498db', '#9b59b6', '#e67e22'][:len(optimizer_stats)],
                     edgecolor='black', linewidth=1.5, width=0.6)

        # Add max and min markers
        ax.scatter(x_pos, optimizer_stats['max'], color='#27ae60', s=150,
                  label='Max mAP@0.5', zorder=5, edgecolors='black', linewidth=1.5, marker='^')
        ax.scatter(x_pos, optimizer_stats['min'], color='#e74c3c', s=150,
                  label='Min mAP@0.5', zorder=5, edgecolors='black', linewidth=1.5, marker='v')

        # Add error bars for standard deviation
        ax.errorbar(x_pos, optimizer_stats['mean'], yerr=optimizer_stats['std'],
                   fmt='none', ecolor='gray', alpha=0.5, capsize=5, capthick=2, linewidth=2)

        ax.set_xlabel('Optimizer Type', fontsize=13, fontweight='bold')
        ax.set_ylabel('mAP@0.5', fontsize=13, fontweight='bold')
        ax.set_title(f'{MODEL_NAME} - Optimizer Performance Comparison (Mean ± Std Dev)',
                    fontsize=14, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels([])
        ax.legend(fontsize=11, loc='lower left', framealpha=0.9, ncol=2)
        ax.grid(True, alpha=0.3, axis='y', linestyle='--')

        # Add optimizer names inside bars
        for i, (opt, row) in enumerate(optimizer_stats.iterrows()):
            # Optimizer name inside bar (centered vertically)
            ax.text(i, row['mean'] / 2, opt.upper(),
                   ha='center', va='center', fontsize=12, fontweight='bold',
                   color='white', rotation=0)

            # Mean value below optimizer name in bar
            ax.text(i, row['mean'] / 2 - 0.02, f"{row['mean']:.4f}",
                   ha='center', va='top', fontsize=9, fontweight='bold',
                   color='white', alpha=0.9)

            # Trial count above max point
            ax.text(i, row['max'] - 0.05, f"n={int(row['count'])}",
                   ha='center', va='bottom', fontsize=10, fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7, edgecolor='black'))

        plt.tight_layout()

        optimizer_comp_img = TUNE_DIR / 'report_optimizer_comparison.png'
        plt.savefig(optimizer_comp_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(optimizer_comp_img), width=6.5*inch, height=3.25*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Optimizer comparison chart saved: {optimizer_comp_img}')

        # Add detailed statistics table for optimizers
        optimizer_table_data = [['Optimizer', 'Mean', 'Max', 'Min', 'Std Dev', 'Trials']]
        for opt, row in optimizer_stats.iterrows():
            optimizer_table_data.append([
                opt.upper(),
                f"{row['mean']:.4f}",
                f"{row['max']:.4f}",
                f"{row['min']:.4f}",
                f"{row['std']:.4f}",
                str(int(row['count']))
            ])

        opt_table = Table(optimizer_table_data, colWidths=[1.2*inch, 1.0*inch, 1.0*inch, 1.0*inch, 1.0*inch, 0.8*inch])
        opt_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#3498db')),
            ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
            ('TOPPADDING', (0, 0), (-1, -1), 6),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
            ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
        ]))
        story.append(opt_table)
        story.append(Spacer(1, 15))

        # Add interpretation text
        best_optimizer = optimizer_stats.index[0]
        best_mean = optimizer_stats.iloc[0]['mean']
        interpretation = f"<b>Analysis:</b> {best_optimizer.upper()} achieved the highest mean performance ({best_mean:.4f}) across {int(optimizer_stats.iloc[0]['count'])} trials. The error bars show the standard deviation, indicating performance consistency."
        story.append(Paragraph(interpretation, styles['Normal']))
        story.append(Spacer(1, 15))
    else:
        story.append(Paragraph('Optimizer data not available for visualization.', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ optimizer column not found or empty')

    # 5.6 Augmentation Parameters vs Performance
    story.append(PageBreak())
    story.append(Paragraph('5.6 Augmentation Parameters Impact', styles['Heading3']))

    # Create 2x2 subplot for key augmentation parameters
    aug_params = ['mixup', 'mosaic', 'degrees', 'scale']
    available_aug_params = [p for p in aug_params if p in completed_trials_df.columns and completed_trials_df[p].notna().any()]

    print(f'   Available augmentation params: {available_aug_params}')

    if len(available_aug_params) >= 2:
        n_plots = min(len(available_aug_params), 4)
        fig, axes = plt.subplots(2, 2, figsize=(10, 8))
        axes = axes.flatten()

        for idx, param in enumerate(available_aug_params[:4]):
            ax = axes[idx]
            scatter = ax.scatter(completed_trials_df[param], completed_trials_df['mAP@0.5'],
                               c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                               s=60, alpha=0.6, edgecolors='black', linewidth=0.5)
            ax.set_xlabel(param, fontsize=10, fontweight='bold')
            ax.set_ylabel('mAP@0.5', fontsize=10, fontweight='bold')
            ax.set_title(f'{param.capitalize()} Impact', fontsize=11, fontweight='bold')
            ax.grid(True, alpha=0.3)

        # Hide unused subplots
        for idx in range(len(available_aug_params), 4):
            axes[idx].axis('off')

        plt.tight_layout()

        aug_impact_img = TUNE_DIR / 'report_augmentation_impact.png'
        plt.savefig(aug_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(aug_impact_img), width=6.5*inch, height=5.2*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Augmentation impact chart saved: {aug_impact_img}')
    else:
        story.append(Paragraph(f'Insufficient augmentation parameter data for visualization. Found: {available_aug_params}', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ Not enough augmentation params available')

    # 5.7 Weight Decay and Momentum vs Performance
    story.append(PageBreak())
    story.append(Paragraph('5.7 Regularization Parameters Impact', styles['Heading3']))

    has_weight_decay = 'weight_decay' in completed_trials_df.columns and completed_trials_df['weight_decay'].notna().any()
    has_momentum = 'momentum' in completed_trials_df.columns and completed_trials_df['momentum'].notna().any()

    if has_weight_decay and has_momentum:
        print(f'   Creating regularization impact chart...')
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

        # Weight Decay
        scatter1 = ax1.scatter(completed_trials_df['weight_decay'], completed_trials_df['mAP@0.5'],
                              c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                              s=80, alpha=0.6, edgecolors='black', linewidth=0.5)
        ax1.set_xlabel('Weight Decay', fontsize=11, fontweight='bold')
        ax1.set_ylabel('mAP@0.5', fontsize=11, fontweight='bold')
        ax1.set_title('Weight Decay Impact', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)

        # Momentum
        scatter2 = ax2.scatter(completed_trials_df['momentum'], completed_trials_df['mAP@0.5'],
                              c=completed_trials_df['mAP@0.5'], cmap='RdYlGn',
                              s=80, alpha=0.6, edgecolors='black', linewidth=0.5)
        ax2.set_xlabel('Momentum', fontsize=11, fontweight='bold')
        ax2.set_ylabel('mAP@0.5', fontsize=11, fontweight='bold')
        ax2.set_title('Momentum Impact', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)

        plt.tight_layout()

        reg_impact_img = TUNE_DIR / 'report_regularization_impact.png'
        plt.savefig(reg_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(reg_impact_img), width=6.5*inch, height=2.6*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Regularization impact chart saved: {reg_impact_img}')
    else:
        story.append(Paragraph(f'Regularization parameter data not available. weight_decay: {has_weight_decay}, momentum: {has_momentum}', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ weight_decay or momentum columns not found or empty')

    # 5.8 Image Size Impact on Performance
    story.append(PageBreak())
    story.append(Paragraph('5.8 Image Size Impact on Performance', styles['Heading3']))

    if 'imgsz' in completed_trials_df.columns and completed_trials_df['imgsz'].notna().any():
        print(f'   Creating image size impact chart...')
        fig, ax = plt.subplots(figsize=(10, 5))

        # Group by image size and calculate statistics
        imgsz_stats = completed_trials_df.groupby('imgsz')['mAP@0.5'].agg(['mean', 'max', 'min', 'count'])
        imgsz_stats = imgsz_stats.sort_index()

        x_pos = range(len(imgsz_stats))
        bars = ax.bar(x_pos, imgsz_stats['mean'], alpha=0.7, color='#9b59b6',
               label='Mean mAP@0.5', edgecolor='black', linewidth=1.5, width=0.6)
        ax.scatter(x_pos, imgsz_stats['max'], color='#27ae60', s=120,
                  label='Max mAP@0.5', zorder=5, edgecolors='black', linewidth=1, marker='^')
        ax.scatter(x_pos, imgsz_stats['min'], color='#e74c3c', s=120,
                  label='Min mAP@0.5', zorder=5, edgecolors='black', linewidth=1, marker='v')

        ax.set_xlabel('Image Size (pixels)', fontsize=12, fontweight='bold')
        ax.set_ylabel('mAP@0.5', fontsize=12, fontweight='bold')
        ax.set_title(f'{MODEL_NAME} - Image Size Impact on Performance', fontsize=14, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels([int(idx) for idx in imgsz_stats.index], fontsize=11, fontweight='bold')
        ax.legend(fontsize=10, loc='best')
        ax.grid(True, alpha=0.3, axis='y')

        # Add count and mean value annotations
        for i, (imgsz, row) in enumerate(imgsz_stats.iterrows()):
            # Mean value inside bar
            ax.text(i, row['mean'] / 2, f"{row['mean']:.4f}",
                   ha='center', va='center', fontsize=10, fontweight='bold', color='white')
            # Count above bar
            ax.text(i, row['max'] + 0.003, f"n={int(row['count'])}",
                   ha='center', va='bottom', fontsize=9)

        plt.tight_layout()

        imgsz_impact_img = TUNE_DIR / 'report_imgsz_impact.png'
        plt.savefig(imgsz_impact_img, dpi=150, bbox_inches='tight')
        plt.close()

        story.append(Image(str(imgsz_impact_img), width=6.5*inch, height=3.25*inch))
        story.append(Spacer(1, 15))
        print(f'✓ Image size impact chart saved: {imgsz_impact_img}')

        # Add statistics table for image sizes
        imgsz_table_data = [['Image Size', 'Mean mAP@0.5', 'Max mAP@0.5', 'Min mAP@0.5', 'Trials']]
        for imgsz, row in imgsz_stats.iterrows():
            imgsz_table_data.append([
                str(int(imgsz)),
                f"{row['mean']:.4f}",
                f"{row['max']:.4f}",
                f"{row['min']:.4f}",
                str(int(row['count']))
            ])

        imgsz_table = Table(imgsz_table_data, colWidths=[1.2*inch, 1.2*inch, 1.2*inch, 1.2*inch, 0.8*inch])
        imgsz_table.setStyle(TableStyle([
            ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#9b59b6')),
            ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
            ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
            ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
            ('FONTSIZE', (0, 0), (-1, 0), 10),
            ('FONTSIZE', (0, 1), (-1, -1), 9),
            ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
            ('TOPPADDING', (0, 0), (-1, -1), 6),
            ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
            ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
        ]))
        story.append(imgsz_table)
        story.append(Spacer(1, 15))
    else:
        story.append(Paragraph('Image size data not available for visualization.', styles['Normal']))
        story.append(Spacer(1, 15))
        print(f'   ⚠️ imgsz column not found or empty')

    print('✓ All custom visualizations generated for PDF report')

# ===== SECTION 5.9: KEY INSIGHTS & RECOMMENDATIONS =====
story.append(PageBreak())
story.append(Paragraph('5.9 Key Insights & Production Recommendations', styles['Heading3']))

print('   Generating key insights and recommendations...')

# Generate comprehensive recommendations based on best trial
best_trial_row = completed_trials_df[completed_trials_df['trial'] == study.best_trial.number].iloc[0]

recommendations_text = f"""
<b>🎯 Optimal Configuration for Production Deployment:</b><br/><br/>

<b>1. Image Processing:</b><br/>
   • Use <b>{int(best_trial_row.get('imgsz', 'N/A'))}px</b> input resolution for optimal accuracy<br/>
   • Expected performance: <b>mAP@0.5 = {study.best_value:.4f}</b><br/>
   • Tradeoff: Higher resolution improves accuracy but increases inference time<br/><br/>

<b>2. Optimizer Configuration:</b><br/>
   • Algorithm: <b>{best_trial_row.get('optimizer', 'N/A')}</b><br/>
   • Learning rate (lr0): <b>{best_trial_row.get('lr0', 0):.6f}</b><br/>
   • Momentum: <b>{best_trial_row.get('momentum', 0):.4f}</b><br/>
   • Weight decay: <b>{best_trial_row.get('weight_decay', 0):.6f}</b><br/><br/>

<b>3. Training Warmup:</b><br/>
   • Warmup epochs: <b>{int(best_trial_row.get('warmup_epochs', 0))}</b><br/>
   • Warmup momentum: <b>{best_trial_row.get('warmup_momentum', 0):.4f}</b><br/>
   • Warmup bias lr: <b>{best_trial_row.get('warmup_bias_lr', 0):.6f}</b><br/><br/>

<b>4. Data Augmentation:</b><br/>
   • Mosaic augmentation: <b>{best_trial_row.get('mosaic', 0):.4f}</b> (strong augmentation for robustness)<br/>
   • Mixup augmentation: <b>{best_trial_row.get('mixup', 0):.4f}</b> (light augmentation)<br/>
   • Recommendation: Use these exact values for similar datasets<br/><br/>

<b>5. Performance Metrics:</b><br/>
   • Best trial found at <b>#{int(best_trial_row['trial'])}</b> out of {len(study.trials)} trials<br/>
   • Performance improvement: <b>{improvement_pct:.1f}%</b> over worst trial<br/>
   • Consistency: Mean mAP@0.5 = {mean_map:.4f} (Std = {completed_df_summary["mAP@0.5"].std():.4f})<br/><br/>

<b>6. Deployment Recommendations:</b><br/>
"""

# Add optimizer-specific recommendations
if 'optimizer' in completed_df_summary.columns:
    opt_comparison = completed_df_summary.groupby('optimizer')['mAP@0.5'].agg(['mean', 'std', 'count'])
    recommendations_text += f"   • <b>{best_opt}</b> optimizer demonstrated best performance (mean: {best_opt_mean:.4f})<br/>"

    if len(opt_comparison) > 1:
        other_opts = opt_comparison[opt_comparison.index != best_opt]
        if len(other_opts) > 0:
            worst_opt = other_opts['mean'].idxmin()
            diff_pct = ((best_opt_mean - other_opts.loc[worst_opt, 'mean']) / other_opts.loc[worst_opt, 'mean']) * 100
            recommendations_text += f"   • <b>{best_opt}</b> outperformed {worst_opt} by {diff_pct:.1f}%<br/>"

# Add image size recommendations
if 'imgsz' in completed_df_summary.columns and len(completed_df_summary['imgsz'].unique()) > 1:
    imgsz_comparison = completed_df_summary.groupby('imgsz')['mAP@0.5'].mean()
    recommendations_text += f"   • For maximum accuracy, use {int(best_imgsz)}px images<br/>"
    if len(imgsz_comparison) > 1:
        smaller_sizes = imgsz_comparison[imgsz_comparison.index < best_imgsz]
        if len(smaller_sizes) > 0:
            recommendations_text += f"   • For faster inference with slight accuracy trade-off, consider {int(smaller_sizes.index[-1])}px (mAP: {smaller_sizes.iloc[-1]:.4f})<br/>"

recommendations_text += f"""<br/>
<b>7. Next Steps:</b><br/>
   • Train full model with these hyperparameters <br/>
   • Monitor validation metrics for overfitting<br/>
   • Consider ensemble methods for further improvement<br/><br/>

<b>📊 Confidence Level:</b><br/>
   • Based on {len(completed_df_summary)} successful trials<br/>
   • Optimization converged {'early' if convergence_pct < 40 else 'steadily'} (best at {convergence_pct:.1f}% through search)<br/>
   • Standard deviation ({completed_df_summary['mAP@0.5'].std():.4f}) indicates {'high' if completed_df_summary['mAP@0.5'].std() < 0.02 else 'moderate'} consistency
"""

story.append(Paragraph(recommendations_text, styles['Normal']))
story.append(Spacer(1, 20))

# Add comparison table: Best vs Mean vs Worst
comparison_data = [
    ['Metric', 'Best Trial', 'Mean Performance', 'Worst Trial'],
    ['mAP@0.5', f'{best_map:.4f}', f'{mean_map:.4f}', f'{worst_map:.4f}'],
    ['Trial #', f'#{int(best_trial_row["trial"])}', '-', f'#{int(completed_df_summary.loc[completed_df_summary["mAP@0.5"].idxmin(), "trial"])}'],
]

# Add key parameters
if 'lr0' in best_trial_row:
    worst_trial_row = completed_df_summary.loc[completed_df_summary['mAP@0.5'].idxmin()]
    comparison_data.append(['Learning Rate', f'{best_trial_row["lr0"]:.6f}',
                           f'{completed_df_summary["lr0"].mean():.6f}', f'{worst_trial_row["lr0"]:.6f}'])
if 'momentum' in best_trial_row:
    comparison_data.append(['Momentum', f'{best_trial_row["momentum"]:.4f}',
                           f'{completed_df_summary["momentum"].mean():.4f}', f'{worst_trial_row["momentum"]:.4f}'])

comparison_table = Table(comparison_data, colWidths=[1.5*inch, 1.5*inch, 1.5*inch, 1.5*inch])
comparison_table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#e74c3c')),
    ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
    ('BACKGROUND', (1, 1), (1, -1), rl_colors.HexColor('#d5f4e6')),  # Highlight best column
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 10),
    ('FONTSIZE', (0, 1), (-1, -1), 9),
    ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
    ('TOPPADDING', (0, 0), (-1, -1), 6),
    ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
]))
story.append(comparison_table)
story.append(Spacer(1, 20))

print('✓ Insights and recommendations section completed')

# ===== SECTION 6: ALL TRIALS SUMMARY =====
story.append(PageBreak())
story.append(Paragraph('6. All Trials Summary', heading_style))

# Statistics
completed_df = df_trials_sorted[df_trials_sorted['state'] == 'COMPLETE']
if len(completed_df) > 0:
    stats_data = [
        ['Metric', 'Value'],
        ['Completed Trials', str(len(completed_df))],
        ['Best mAP@0.5', f"{completed_df['mAP@0.5'].max():.4f}"],
        ['Worst mAP@0.5', f"{completed_df['mAP@0.5'].min():.4f}"],
        ['Mean mAP@0.5', f"{completed_df['mAP@0.5'].mean():.4f}"],
        ['Std Dev mAP@0.5', f"{completed_df['mAP@0.5'].std():.4f}"],
        ['Median mAP@0.5', f"{completed_df['mAP@0.5'].median():.4f}"],
    ]

    stats_table = Table(stats_data, colWidths=[2.5*inch, 3.5*inch])
    stats_table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), rl_colors.HexColor('#e74c3c')),
        ('TEXTCOLOR', (0, 0), (-1, 0), rl_colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, -1), 10),
        ('BOTTOMPADDING', (0, 0), (-1, -1), 6),
        ('TOPPADDING', (0, 0), (-1, -1), 6),
        ('ROWBACKGROUNDS', (0, 1), (-1, -1), [rl_colors.white, rl_colors.lightgrey]),
        ('GRID', (0, 0), (-1, -1), 1, rl_colors.black)
    ]))
    story.append(stats_table)

# Build PDF
try:
    doc.build(story)
    print(f'\n✓ Comprehensive PDF report generated: {pdf_report_path}')
    print(f'  Size: {pdf_report_path.stat().st_size / (1024*1024):.1f} MB')
    print(f'  Sections: Overview, Executive Summary, Best Hyperparameters, Top 20 Trials,')
    print(f'            Performance Analysis (10 advanced visualizations), Key Insights,')
    print(f'            Production Recommendations, All Trials Summary')
    print(f'  Charts: Distribution, Correlation, Timeline, mAP Progress, Learning Rate,')
    print(f'          Optimizer, Augmentation, Regularization, Image Size')
except Exception as pdf_error:
    print(f'\n⚠️  Error generating PDF: {pdf_error}')
    import traceback
    traceback.print_exc()

print('=' * 80)


GENERATING COMPREHENSIVE TUNING PDF REPORT

📊 Preparing comprehensive report with 42 trials
   Best Trial: 35
   Best mAP@0.5: 0.5771

📋 Compiling trials data for report...
✓ Compiled 42 trials for report
   Available columns: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'warmup_epochs', 'warmup_momentum', 'warmup_bias_lr', 'mosaic', 'mixup']
✓ Executive summary generated
   DataFrame columns: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'warmup_epochs', 'warmup_momentum', 'warmup_bias_lr', 'mosaic', 'mixup']
   Sample row keys: ['trial', 'state', 'mAP@0.5', 'imgsz', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'warmup_epochs', 'warmup_momentum', 'warmup_bias_lr', 'mosaic', 'mixup']
   Creating detailed params for top 5 trials...
   ✓ Top 5 trials details added

📊 Generating custom visualizations for PDF report...
   Completed trials: 41
   Columns available: ['trial', 'state', 'mAP@0.5', 'imgsz', '

## 13. Analyze Best Hyperparameters

In [17]:
# DISPLAY BEST HYPERPARAMETERS
# ============================================================================

print('\n' + '=' * 80)
print('BEST HYPERPARAMETERS')
print('=' * 80)

print(f'\nBest Trial Number: {study.best_trial.number}')
print(f'Best mAP@0.5: {study.best_value:.4f}')
print('\nOptimized Hyperparameters:')
print(json.dumps(study.best_params, indent=2))
print('=' * 80)


BEST HYPERPARAMETERS

Best Trial Number: 35
Best mAP@0.5: 0.5771

Optimized Hyperparameters:
{
  "imgsz": 768,
  "optimizer": "SGD",
  "lr0": 0.0001614596240725037,
  "momentum": 0.9098500376932075,
  "weight_decay": 0.00042786651935909105,
  "warmup_epochs": 1,
  "warmup_momentum": 0.5149307497845836,
  "warmup_bias_lr": 0.018938388657529233,
  "mosaic": 0.8205379408033115,
  "mixup": 0.023789168654792303
}


## 13. Create Trials Summary

In [18]:
# CREATE TRIALS SUMMARY AND DATAFRAME (SHARED RESOURCE)
# ============================================================================

print('\n' + '=' * 80)
print('TRIALS SUMMARY')
print('=' * 80)

# Compile all trial data (used by multiple sections)
trials_data = []
for trial in study.trials:
    trial_info = {
        'trial': trial.number,
        'mAP@0.5': trial.value if trial.value else 0.0,
        'state': trial.state.name,
        'duration_seconds': (trial.datetime_complete - trial.datetime_start).total_seconds() if trial.datetime_complete else None,
    }
    # Add all parameters
    trial_info.update(trial.params)
    trials_data.append(trial_info)

# Create DataFrame and sort by performance (used by PDF report and display)
df_trials = pd.DataFrame(trials_data)
df_trials_sorted = df_trials.sort_values('mAP@0.5', ascending=False)

print('\n📊 TOP 10 TRIALS:')
print('=' * 80)
# Display top 10 with selected columns
display_cols = ['trial', 'mAP@0.5', 'state', 'optimizer', 'lr0', 'momentum', 'weight_decay', 'mixup']
available_cols = [col for col in display_cols if col in df_trials_sorted.columns]
print(df_trials_sorted[available_cols].head(10).to_string(index=False))
print('=' * 80)

# Save complete trials summary
trials_csv_path = TUNE_DIR / 'trials_summary.csv'
df_trials_sorted.to_csv(trials_csv_path, index=False)
print(f'\n✓ Complete trials summary saved to: {trials_csv_path}')

# Save study object
study_path = TUNE_DIR / 'optuna_study.pkl'
with open(study_path, 'wb') as f:
    pickle.dump(study, f)
print(f'✓ Optuna study object saved to: {study_path}')

print('=' * 80)


TRIALS SUMMARY

📊 TOP 10 TRIALS:
 trial  mAP@0.5    state optimizer      lr0  momentum  weight_decay    mixup
    35 0.577074 COMPLETE       SGD 0.000161  0.909850      0.000428 0.023789
     0 0.576928 COMPLETE       SGD 0.000184  0.856970      0.000540 0.166489
    22 0.576661 COMPLETE       SGD 0.000141  0.909425      0.000731 0.123344
    11 0.576523 COMPLETE       SGD 0.000192  0.925614      0.000922 0.189244
    34 0.576213 COMPLETE       SGD 0.000153  0.906332      0.000937 0.038448
    33 0.576136 COMPLETE       SGD 0.000119  0.895453      0.000932 0.079787
    10 0.575980 COMPLETE       SGD 0.000101  0.905713      0.000853 0.143467
    25 0.575915 COMPLETE       SGD 0.000103  0.856550      0.000692 0.129994
    18 0.575635 COMPLETE       SGD 0.000238  0.969074      0.000435 0.113910
    27 0.575601 COMPLETE       SGD 0.000235  0.912563      0.000426 0.163392

✓ Complete trials summary saved to: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/y

## 14. Final summary


In [19]:
# FINAL SUMMARY
# ============================================================================

print('\n\n')
print('=' * 80)
print('HYPERPARAMETER OPTIMIZATION COMPLETE!')
print('=' * 80)

print(f'\n📊 Project: {MODEL_NAME} on {YOLO_DATASET_ROOT.name}')
print(f'📅 Date: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

print(f'\n🔬 Optimization Summary:')
print(f'  Total Trials: {len(study.trials)}')
print(f'  Completed: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}')
print(f'  Best Trial: {study.best_trial.number}')
print(f'  Best Trial mAP@0.5: {study.best_value:.4f}')
print(f'  Duration: {duration}')

if 'final_metrics' in globals():
    print(f'\n🎯 Final Model Performance:')
    print(f'  mAP@0.5: {final_metrics["map50"]:.4f}')
    print(f'  mAP@0.5:0.95: {final_metrics["map50_95"]:.4f}')
    print(f'  Precision: {final_metrics["precision"]:.4f}')
    print(f'  Recall: {final_metrics["recall"]:.4f}')

print(f'\n📁 Generated Files:')
print(f'\n  📊 Tuning Results (in {TUNE_DIR}):')
print(f'    - best_hyperparameters.json')
print(f'    - best_hyperparameters.yaml')
print(f'    - trials_summary.csv')
print(f'    - optuna_study.pkl')
print(f'  📈 Tuning Visualizations:')
print(f'    - optimization_history.html / .png')
print(f'    - parameter_importance.html / .png')
print(f'    - parameter_slice.html / .png')
print(f'  📄 Tuning PDF Report:')
print(f'    - {MODEL_NAME}_tuning_report.pdf')

print(f'\n📂 All results saved to:')
print(f'  Tuning: {TUNE_DIR}')

print(f'\n🎓 Top 5 Hyperparameters (by importance):')
try:
    importances = optuna.importance.get_param_importances(study)
    for i, (param, importance) in enumerate(list(importances.items())[:5], 1):
        print(f'  {i}. {param}: {importance:.4f}')
except:
    print('  (Not available - requires completed trials with variation)')

print(f'\n🚀 Next Steps:')
print(f'  1. Review tuning PDF report: {TUNE_DIR / f"{MODEL_NAME}_tuning_report.pdf"}')
print(f'  2. Review optimization visualizations in: {TUNE_DIR}')
print(f'  3. Use best_hyperparameters.yaml for training in a separate notebook')

print('\n' + '=' * 80)
print('SUCCESS! ✓')
print('=' * 80)




HYPERPARAMETER OPTIMIZATION COMPLETE!

📊 Project: yolov8m_finetuned_1 on bdd100k_yolo_tuning
📅 Date: 2025-11-28 20:18:43

🔬 Optimization Summary:
  Total Trials: 42
  Completed: 41
  Best Trial: 35
  Best Trial mAP@0.5: 0.5771
  Duration: 2:37:27.650793

📁 Generated Files:

  📊 Tuning Results (in /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340):
    - best_hyperparameters.json
    - best_hyperparameters.yaml
    - trials_summary.csv
    - optuna_study.pkl
  📈 Tuning Visualizations:
    - optimization_history.html / .png
    - parameter_importance.html / .png
    - parameter_slice.html / .png
  📄 Tuning PDF Report:
    - yolov8m_finetuned_1_tuning_report.pdf

📂 All results saved to:
  Tuning: /content/Drive/MyDrive/ksu_yolo_tuning_2025/computer_vision_yolo/tune_train/tune/yolov8m_finetuned_1_tune_20251127_230340

🎓 Top 5 Hyperparameters (by importance):
  1. imgsz: 0.7685
  2. optimizer: 0.1955
  3. weight_deca